In [1]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD')

In [2]:
try:
    con = pymysql.connect(
    host='localhost',
    user='root',
    password=password,
    charset='utf8mb4'
    )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")


Connection to the database was successful!


In [3]:
# calling this function makes life easy and just need to pass the name of the database I want to make a query on
# It serves like USE keyword in SQL

def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

In [4]:
# to get my tables' names in the database

def table_names (database):
    
    con = connector(database)
    
    query = "SELECT table_name FROM information_schema.tables WHERE table_schema='{}';".format(database)

    # Execute the query and store the results in a Pandas DataFrame
    tables = pd.read_sql_query(query, con)

    # Print the list of tables
    print(tables)

In [5]:
# to get my views' names in the database

def views_names (database):
    
    con = connector(database)
    
    query = "SELECT table_name FROM information_schema.views WHERE table_schema='{}';".format(database)

    # Execute the query and store the results in a Pandas DataFrame
    tables = pd.read_sql_query(query, con)

    # Print the list of tables
    print(tables)

In [6]:
# to get the names of stored procedures in the database

def stored_procedures_names (database):
    
    con = connector(database)
    
    #query = "SELECT table_name FROM information_schema.views WHERE table_schema='{}';".format(database)
    query = "SELECT specific_name FROM information_schema.routines WHERE routine_type='PROCEDURE' AND routine_schema='{}';".format(database)
    
    # Execute the query and store the results in a Pandas DataFrame
    tables = pd.read_sql_query(query, con)

    # Print the list of tables
    print(tables)

In [19]:
table_names('sql_store')

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [6]:
# How to get the names of all the databases in my MySQL server 

# I can use the cursor object from my database connection to execute SQL commands and retrieve information 
# about my databases. Here's an example of how I can use the **SHOW DATABASES** SQL command to retrieve the 
# names of all databases in my MySQL server using my connector function:


# Create a database connection
conn = connector(database='sql_store')

# Create a cursor object
cursor = conn.cursor()

# Execute the SHOW DATABASES command
cursor.execute('SHOW DATABASES')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the names of all databases in the MySQL server
for row in results:
    print(row[0])

# Close the cursor and connection
cursor.close()
conn.close()


Leetcode_Q_1050
Leetcode_Q_1068
Leetcode_Q_1075
Leetcode_Q_1084
Leetcode_Q_1141
Leetcode_Q_1148
Leetcode_Q_1179
Leetcode_Q_1211
Leetcode_Q_1251
Leetcode_Q_1280
Leetcode_Q_1327
Leetcode_Q_1378
Leetcode_Q_1407
Leetcode_Q_1484
Leetcode_Q_1517
Leetcode_Q_1527
Leetcode_Q_1581
Leetcode_Q_1587
Leetcode_Q_1633
Leetcode_Q_1661
Leetcode_Q_1667
Leetcode_Q_1683
Leetcode_Q_1693
Leetcode_Q_1729
Leetcode_Q_1731
Leetcode_Q_1741
Leetcode_Q_1757
Leetcode_Q_1789
Leetcode_Q_1795
Leetcode_Q_181
Leetcode_Q_182
Leetcode_Q_183
Leetcode_Q_1873
Leetcode_Q_1890
Leetcode_Q_196
Leetcode_Q_1965
Leetcode_Q_197
Leetcode_Q_1978
Leetcode_Q_2356
Leetcode_Q_511
Leetcode_Q_577
Leetcode_Q_584
Leetcode_Q_586
Leetcode_Q_595
Leetcode_Q_596
Leetcode_Q_607
Leetcode_Q_610
Leetcode_Q_619
Leetcode_Q_620
Leetcode_Q_627
information_schema
invoicing
mysql
performance_schema
sql_hr
sql_inventory
sql_invoicing
sql_store
sys


In [7]:
# how to get column attributes

def column_attribute (database, table_name):
    
    con = connector(database)
    
    query = """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{}'
    """.format(table_name)

    # read the query result into a DataFrame
    df = pd.read_sql(query, con)
    return df

In [8]:
# to get the default values for each column

def default_column_values (database, col_name):
    
    con = connector(database)
    
    query = '''
        SELECT COLUMN_NAME, IF(IS_NULLABLE = 'YES', 'NULL', COLUMN_DEFAULT) AS COLUMN_DEFAULT
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = '{}'
         '''.format(col_name)


    # Execute the query and store the results in a pandas DataFrame
    df = pd.read_sql(query, con)
    
   # Print the default values for each column
    for idx, row in df.iterrows():
        col_name = row['COLUMN_NAME']
        col_default = row['COLUMN_DEFAULT']
        if col_default is None:
            print(f"Column {col_name} does not have a default value")
        elif str(col_default).upper() == 'NULL':
            print(f"Column {col_name} has a NULL default value")
        else:
            print(f"Column {col_name} has default value: {col_default}")


In [9]:
# to get the name of the primary keys of tables within a database

# I can use the cursor object from my database connection to execute SQL commands and retrieve 
# information about the primary keys of tables in my database. Here's my function to 
# use the INFORMATION_SCHEMA database to retrieve information about the primary keys of tables in my 
# MySQL server using my connector function:

def primary_key_finder(database):
    
    con = connector(database)
    
    # Create a cursor object
    cursor = con.cursor()

    # Execute the SQL command to retrieve primary key information
    cursor.execute('''
        SELECT kcu.table_name, kcu.column_name, kcu.constraint_name
        FROM information_schema.table_constraints tco
        JOIN information_schema.key_column_usage kcu
          ON kcu.constraint_name = tco.constraint_name
          AND kcu.constraint_schema = tco.constraint_schema
          AND kcu.table_name = tco.table_name
        WHERE tco.constraint_type = 'PRIMARY KEY'
          AND tco.table_schema = %s
        ORDER BY kcu.table_name, kcu.ordinal_position;''', (database,))


    # Fetch all the results as a list of tuples
    results = cursor.fetchall()

    # Print the primary key information
    for row in results:
        print(row[0], row[1], row[2])

    # Close the cursor and connection
    cursor.close()
    con.close()


In [306]:
primary_key_finder('sql_store')

customers customer_id PRIMARY
order_item_notes note_id PRIMARY
order_items order_id PRIMARY
order_items product_id PRIMARY
order_statuses order_status_id PRIMARY
orders order_id PRIMARY
products product_id PRIMARY
shippers shipper_id PRIMARY


In [ ]:
# To insert a row: 

# pd.read_sql() is used to execute a SELECT query and return the results as a pandas DataFrame.

# I need to execute an INSERT query. Like:

con = connector('sql_store')
cursor = con.cursor()

cursor.execute("""
    INSERT INTO customers (first_nam) 
    VALUES ('John')
""")

con.commit()

In [10]:
def delete_database(database):
    conn = connector(database=None)
    cursor = conn.cursor()

    # Replace <database_name> with the name of the database you want to delete
    database_name = database

    # Execute the DROP DATABASE SQL command
    cursor.execute(f"DROP DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()

In [11]:
def delete_table(database, table_name):
    
    con = connector(database)
    cursor = con.cursor()
    
    cursor.execute(f"DROP TABLE {table_name}")
    
    # Commit the transaction
    con.commit()

    # Close the connection
    con.close()

In [12]:
def delete_view (database, view_name):
    
    con = connector(database)
    cursor = con.cursor()
    
    cursor.execute(f"DROP VIEW {view_name}")
    
    # Commit the transaction
    con.commit()

    # Close the connection
    con.close()

In [61]:
def delete_stored_procedure(database, stored_procedure_name):
    
    con = connector(database)
    cursor = con.cursor()
    
    cursor.execute(f"DROP PROCEDURE IF EXISTS {stored_procedure_name}")
    
    # Commit the transaction
    con.commit()

    # Close the connection
    con.close()

# Retrieving data from a single table

# SELECT

In [8]:
con = connector ('sql_inventory')
df = pd.read_sql("SELECT * FROM products", con)
df


,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [38]:
con = connector ('sql_store')

In [39]:
df = pd.read_sql("SELECT * \
                 FROM customers \
                 WHERE birth_date>'1990-01-01'\
                 ORDER BY last_name", 
                 con)
df


,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [46]:
df = pd.read_sql("SELECT first_name, last_name \
                    FROM customers", 
                    con)
df


,first_name,last_name
0,Babara,MacCaffrey
1,Ines,Brushfield
2,Freddi,Boagey
3,Ambur,Roseburgh
4,Clemmie,Betchley
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Romola,Rumgay
9,Levy,Mynett


In [47]:
df = pd.read_sql("SELECT last_name, first_name \
                    FROM customers", 
                    con)
df


,last_name,first_name
0,MacCaffrey,Babara
1,Brushfield,Ines
2,Boagey,Freddi
3,Roseburgh,Ambur
4,Betchley,Clemmie
5,Twiddell,Elka
6,Dowson,Ilene
7,Naseby,Thacher
8,Rumgay,Romola
9,Mynett,Levy


In [67]:
pd.read_sql("SELECT first_name, last_name, points, points*10 AS '10x POINTS' \
            FROM customers", con)

,first_name,last_name,points,10x POINTS
0,Babara,MacCaffrey,2273,22730
1,Ines,Brushfield,947,9470
2,Freddi,Boagey,2967,29670
3,Ambur,Roseburgh,457,4570
4,Clemmie,Betchley,3675,36750
5,Elka,Twiddell,3073,30730
6,Ilene,Dowson,1672,16720
7,Thacher,Naseby,205,2050
8,Romola,Rumgay,1486,14860
9,Levy,Mynett,796,7960


In [76]:
pd.read_sql("SELECT DISTINCT state FROM customers", con)

,state
0,MA
1,VA
2,CO
3,FL
4,TX
5,IL
6,TN
7,CA
8,GA


In [77]:
con = connector ('sql_inventory')

In [81]:
pd.read_sql("SELECT name, unit_price, unit_price * 1.1 AS 'new price' \
            FROM products", con)

,name,unit_price,new price
0,Foam Dinner Plate,1.21,1.331
1,"Pork - Bacon,back Peameal",4.65,5.115
2,"Lettuce - Romaine, Heart",3.35,3.685
3,"Brocolinni - Gaylan, Chinese",4.53,4.983
4,Sauce - Ranch Dressing,1.63,1.793
5,Petit Baguette,2.39,2.629
6,Sweet Pea Sprouts,3.29,3.619
7,Island Oasis - Raspberry,0.74,0.814
8,Longan,2.26,2.486
9,Broom - Push,1.09,1.199


In [82]:
con = connector('sql_store')

# WHERE

In [83]:
pd.read_sql("SELECT * FROM customers WHERE points>3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [87]:
pd.read_sql('SELECT * FROM customers WHERE state != "VA"', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [89]:
pd.read_sql("SELECT * FROM customers WHERE birth_date > '1990-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [3]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [98]:
pd.read_sql("SELECT * FROM orders WHERE order_date>='2019-01-01'", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None


In [6]:
con = connector('sql_store')

# AND OR NOT

In [13]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' AND points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [14]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
6,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [17]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR \
            (points>1000 AND state ='VA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [21]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE NOT (birth_date > '1990-01-01' OR points>1000)", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [23]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE  birth_date <= '1990-01-01' AND points<=1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
con = connector('sql_store')

pd.read_sql("SELECT * \
            FROM order_items \
            WHERE (order_id = 6 AND (unit_price * quantity)> 30)", con)

,order_id,product_id,quantity,unit_price
0,6,1,4,8.65


In [34]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state ='VA' OR state = 'GA' OR state = 'FL'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# IN

In [35]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [36]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state NOT IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [39]:
pd.read_sql("SELECT *\
            FROM products\
            WHERE quantity_in_stock IN (49, 38, 72)", con)

,product_id,name,quantity_in_stock,unit_price
0,2,"Pork - Bacon,back Peameal",49,4.65
1,3,"Lettuce - Romaine, Heart",38,3.35


In [40]:
pd.read_sql("SELECT *\
            FROM customers\
                WHERE points >=1000 AND points <=3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# BETWEEN

In [43]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE points BETWEEN 1000 AND 3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [48]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE birth_date BETWEEN '1990-01-01' AND '2000-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# LIKE

In [49]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [50]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'brush%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [51]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [52]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points


In [55]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [56]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [69]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE address LIKE '%TRAIL%' OR \
                  address LIKE '%AVENUE%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [70]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [71]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone NOT LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
6,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [4]:
con = connector('sql_store')

In [5]:
pd.read_sql('SELECT * FROM customers', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# REGEXP

In [6]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%field%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [7]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [8]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^Brush'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [9]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [14]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [15]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [16]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [17]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [23]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[gim]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [25]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'e[l]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [26]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[a-h]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [27]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE first_name REGEXP 'ELKA|AMBUR'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [28]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'EY$|ON$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [29]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^MY|SE'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
pd.read_sql("SELECT * \
            FROM customers\
            WHERE last_name REGEXP 'B[RU]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


# IS NULL

In [35]:
pd.read_sql("SELECT * \
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [38]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [39]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NOT NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [40]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [48]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [52]:
pd.read_sql("SELECT *\
            FROM orders\
            WHERE shipped_date IS NULL", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None
1,3,8,2017-12-01,1,None,None,None
2,4,2,2017-01-22,1,None,None,None
3,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,None
4,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,None


# ORDER BY

In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [54]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name DESC", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
3,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
6,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
7,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
8,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
9,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [55]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
7,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
8,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
9,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [56]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state DESC, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
6,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
9,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [58]:
pd.read_sql("SELECT first_name, last_name\
            FROM customers\
            ORDER BY birth_date", con)

,first_name,last_name
0,Ilene,Dowson
1,Levy,Mynett
2,Clemmie,Betchley
3,Ambur,Roseburgh
4,Freddi,Boagey
5,Babara,MacCaffrey
6,Ines,Brushfield
7,Elka,Twiddell
8,Romola,Rumgay
9,Thacher,Naseby


In [60]:
pd.read_sql("SELECT first_name, last_name, 10 AS points\
            FROM customers\
            ORDER BY points, birth_date", con)

,first_name,last_name,points
0,Ilene,Dowson,10
1,Levy,Mynett,10
2,Clemmie,Betchley,10
3,Ambur,Roseburgh,10
4,Freddi,Boagey,10
5,Babara,MacCaffrey,10
6,Ines,Brushfield,10
7,Elka,Twiddell,10
8,Romola,Rumgay,10
9,Thacher,Naseby,10


In [72]:
pd.read_sql("SELECT *, quantity * unit_price AS total_price\
            FROM order_items\
            WHERE order_id = 2\
            ORDER BY total_price DESC", con)

,order_id,product_id,quantity,unit_price,total_price
0,2,1,2,9.10,18.20
1,2,4,4,1.66,6.64
2,2,6,2,2.94,5.88


# LIMIT

In [75]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 4", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [76]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 400", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [79]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 6, 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [82]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY points DESC\
            LIMIT 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


# Retrieving data from multiple tables

# INNER JOIN

In [7]:
con = connector('sql_store')

In [8]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [10]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [12]:
pd.read_sql("SELECT *\
            FROM orders\
            JOIN customers \
            ON orders.customer_id = customers.customer_id", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,6,2019-01-30,1,None,None,NaN,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,2,7,2018-08-02,2,None,2018-08-03,4.0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
2,3,8,2017-12-01,1,None,None,NaN,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,4,2,2017-01-22,1,None,None,NaN,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
4,5,5,2017-08-25,2,,2017-08-26,3.0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
6,7,2,2018-09-22,2,None,2018-09-23,4.0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
9,10,6,2018-04-22,2,None,2018-04-23,2.0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [15]:
pd.read_sql("SELECT order_id, orders.customer_id, first_name, last_name\
            FROM orders\
            JOIN customers \
                ON orders.customer_id = customers.customer_id", con)

,order_id,customer_id,first_name,last_name
0,4,2,Ines,Brushfield
1,7,2,Ines,Brushfield
2,5,5,Clemmie,Betchley
3,8,5,Clemmie,Betchley
4,1,6,Elka,Twiddell
5,10,6,Elka,Twiddell
6,2,7,Ilene,Dowson
7,3,8,Thacher,Naseby
8,6,10,Levy,Mynett
9,9,10,Levy,Mynett


In [19]:
pd.read_sql("SELECT order_id, o.customer_id, first_name, last_name\
            FROM orders o\
            JOIN customers c \
                ON o.customer_id = c.customer_id", con)

,order_id,customer_id,first_name,last_name
0,4,2,Ines,Brushfield
1,7,2,Ines,Brushfield
2,5,5,Clemmie,Betchley
3,8,5,Clemmie,Betchley
4,1,6,Elka,Twiddell
5,10,6,Elka,Twiddell
6,2,7,Ilene,Dowson
7,3,8,Thacher,Naseby
8,6,10,Levy,Mynett
9,9,10,Levy,Mynett


In [22]:
pd.read_sql("SELECT *\
            FROM order_items", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


In [23]:
pd.read_sql("SELECT *\
            FROM products", con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [30]:
pd.read_sql("SELECT order_id, oi.product_id, quantity, oi.unit_price\
            FROM order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price
0,2,1,2,9.10
1,6,1,4,8.65
2,10,1,10,6.01
3,5,2,3,9.89
4,6,2,4,3.28
5,3,3,10,9.12
6,4,3,7,6.99
7,6,3,4,7.46
8,7,3,7,9.17
9,1,4,4,3.74


In [31]:
pd.read_sql("SELECT order_id, oi.product_id, quantity, oi.unit_price\
            FROM order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id\
                LIMIT 6, 3", con)

,order_id,product_id,quantity,unit_price
0,4,3,7,6.99
1,6,3,4,7.46
2,7,3,7,9.17


# JOINING ACROSS DATABASES

In [43]:

# Create a database connection
conn = connector(database='sql_store')

# Create a cursor object
cursor = conn.cursor()

# Execute the SHOW DATABASES command
cursor.execute('SHOW DATABASES')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the names of all databases in the MySQL server
for row in results:
    print(row[0])

# Close the cursor and connection
cursor.close()
conn.close()


information_schema
mysql
performance_schema
sql_hr
sql_inventory
sql_invoicing
sql_store
sys


In [44]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN sql_inventory.products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price,product_id,name,quantity_in_stock,unit_price
0,2,1,2,9.10,1,Foam Dinner Plate,70,1.21
1,6,1,4,8.65,1,Foam Dinner Plate,70,1.21
2,10,1,10,6.01,1,Foam Dinner Plate,70,1.21
3,5,2,3,9.89,2,"Pork - Bacon,back Peameal",49,4.65
4,6,2,4,3.28,2,"Pork - Bacon,back Peameal",49,4.65
5,3,3,10,9.12,3,"Lettuce - Romaine, Heart",38,3.35
6,4,3,7,6.99,3,"Lettuce - Romaine, Heart",38,3.35
7,6,3,4,7.46,3,"Lettuce - Romaine, Heart",38,3.35
8,7,3,7,9.17,3,"Lettuce - Romaine, Heart",38,3.35
9,1,4,4,3.74,4,"Brocolinni - Gaylan, Chinese",90,4.53


In [45]:
con = connector('sql_inventory')

pd.read_sql("SELECT *\
            FROM sql_store.order_items oi\
            JOIN products p\
                ON oi.product_id = p.product_id", con)

,order_id,product_id,quantity,unit_price,product_id,name,quantity_in_stock,unit_price
0,2,1,2,9.10,1,Foam Dinner Plate,70,1.21
1,6,1,4,8.65,1,Foam Dinner Plate,70,1.21
2,10,1,10,6.01,1,Foam Dinner Plate,70,1.21
3,5,2,3,9.89,2,"Pork - Bacon,back Peameal",49,4.65
4,6,2,4,3.28,2,"Pork - Bacon,back Peameal",49,4.65
5,3,3,10,9.12,3,"Lettuce - Romaine, Heart",38,3.35
6,4,3,7,6.99,3,"Lettuce - Romaine, Heart",38,3.35
7,6,3,4,7.46,3,"Lettuce - Romaine, Heart",38,3.35
8,7,3,7,9.17,3,"Lettuce - Romaine, Heart",38,3.35
9,1,4,4,3.74,4,"Brocolinni - Gaylan, Chinese",90,4.53


# SELF JOIN

In [46]:
con = connector('sql_hr')

In [56]:
pd.read_sql("SELECT *\
            FROM employees\
            LIMIT 7", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1
3,40448,Mindy,Crissil,Staff Scientist,94860,37270.0,1
4,56274,Keriann,Alloisi,VP Marketing,110150,37270.0,1
5,63196,Alaster,Scutchin,Assistant Professor,32179,37270.0,2
6,67009,North,de Clerc,VP Product Management,114257,37270.0,2


In [55]:
pd.read_sql("SELECT *\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
1,37851,Sayer,Matterson,Statistician III,98926,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
2,40448,Mindy,Crissil,Staff Scientist,94860,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
3,56274,Keriann,Alloisi,VP Marketing,110150,37270,1,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
4,63196,Alaster,Scutchin,Assistant Professor,32179,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
5,67009,North,de Clerc,VP Product Management,114257,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
6,67370,Elladine,Rising,Social Worker,96767,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
7,68249,Nisse,Voysey,Financial Advisor,52832,37270,2,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
8,72540,Guthrey,Iacopetti,Office Assistant I,117690,37270,3,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10
9,72913,Kass,Hefferan,Computer Systems Analyst IV,96401,37270,3,37270,Yovonnda,Magrannell,Executive Secretary,63996,None,10


In [57]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name,\
                m.first_name AS 'Manager first name'\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,first_name,Manager first name
0,33391,D'arcy,Yovonnda
1,37851,Sayer,Yovonnda
2,40448,Mindy,Yovonnda
3,56274,Keriann,Yovonnda
4,63196,Alaster,Yovonnda
5,67009,North,Yovonnda
6,67370,Elladine,Yovonnda
7,68249,Nisse,Yovonnda
8,72540,Guthrey,Yovonnda
9,72913,Kass,Yovonnda


# JOINING MULTIPLE TABLES

In [58]:
con = connector('sql_store')

In [60]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [59]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [61]:
pd.read_sql("SELECT *\
            FROM order_statuses", con)

,order_status_id,name
0,1,Processed
1,2,Shipped
2,3,Delivered


In [62]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [63]:
# join orders table with 2 tables: customers table and order_statuses tables

In [66]:
pd.read_sql("SELECT *\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id,customer_id,first_name,last_name,birth_date,phone,address,city,state,points,order_status_id,name
0,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675,1,Processed
1,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796,1,Processed
2,4,2,2017-01-22,1,None,None,NaN,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947,1,Processed
3,3,8,2017-12-01,1,None,None,NaN,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205,1,Processed
4,1,6,2019-01-30,1,None,None,NaN,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073,1,Processed
5,10,6,2018-04-22,2,None,2018-04-23,2.0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073,2,Shipped
6,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796,2,Shipped
7,7,2,2018-09-22,2,None,2018-09-23,4.0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947,2,Shipped
8,5,5,2017-08-25,2,,2017-08-26,3.0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675,2,Shipped
9,2,7,2018-08-02,2,None,2018-08-03,4.0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672,2,Shipped


In [72]:
pd.read_sql("SELECT \
                o.order_id,\
                o.order_date,\
                c.first_name,\
                c.last_name,\
                os.name AS status\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,order_date,first_name,last_name,status
0,8,2018-06-08,Clemmie,Betchley,Processed
1,6,2018-11-18,Levy,Mynett,Processed
2,4,2017-01-22,Ines,Brushfield,Processed
3,3,2017-12-01,Thacher,Naseby,Processed
4,1,2019-01-30,Elka,Twiddell,Processed
5,10,2018-04-22,Elka,Twiddell,Shipped
6,9,2017-07-05,Levy,Mynett,Shipped
7,7,2018-09-22,Ines,Brushfield,Shipped
8,5,2017-08-25,Clemmie,Betchley,Shipped
9,2,2018-08-02,Ilene,Dowson,Shipped


In [73]:
con = connector('sql_invoicing')

In [74]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_invoicing';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [76]:
pd.read_sql("SELECT *\
            FROM clients", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [77]:
pd.read_sql("SELECT *\
            FROM payment_methods", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal
3,4,Wire Transfer


In [78]:
pd.read_sql("SELECT *\
            FROM payments", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [87]:
pd.read_sql("SELECT \
                p.date,\
                p.invoice_id,\
                p.amount,\
                c.name,\
                pm.name AS 'Payment method'\
            FROM clients c\
            JOIN payments p\
                ON c.client_id = p.client_id\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,invoice_id,amount,name,Payment method
0,2019-02-12,2,8.18,Topiclounge,Credit Card
1,2019-01-03,6,74.55,Vinte,Credit Card
2,2019-01-11,11,0.03,Yadel,Credit Card
3,2019-01-26,13,87.44,Topiclounge,Credit Card
4,2019-01-15,15,80.31,Yadel,Credit Card
5,2019-01-15,17,68.10,Yadel,Credit Card
6,2019-01-08,18,32.77,Topiclounge,Credit Card
7,2019-01-08,18,10.00,Topiclounge,Cash


# COMPOUND JOIN CONDITIONS

In [140]:
# let's get the primary keys of each table

database = 'sql_store'

# Create a database connection
conn = connector(database)

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL command to retrieve primary key information
cursor.execute('''
    SELECT kcu.table_name, kcu.column_name, kcu.constraint_name
    FROM information_schema.table_constraints tco
    JOIN information_schema.key_column_usage kcu
      ON kcu.constraint_name = tco.constraint_name
      AND kcu.constraint_schema = tco.constraint_schema
      AND kcu.table_name = tco.table_name
    WHERE tco.constraint_type = 'PRIMARY KEY'
      AND tco.table_schema = %s
    ORDER BY kcu.table_name, kcu.ordinal_position;''', (database,))


# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the primary key information
for row in results:
    print(row[0], row[1], row[2])

# Close the cursor and connection
cursor.close()
conn.close()


customers customer_id PRIMARY
order_item_notes note_id PRIMARY
order_items order_id PRIMARY
order_items product_id PRIMARY
order_statuses order_status_id PRIMARY
orders order_id PRIMARY
products product_id PRIMARY
shippers shipper_id PRIMARY


In [136]:
con = connector('sql_store')

In [133]:
pd.read_sql("SELECT *\
            FROM order_items", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


In [142]:
pd.read_sql("SELECT *\
            FROM order_item_notes", con)

,note_id,order_Id,product_id,note
0,1,2,1,first note
1,2,2,1,second note


In [141]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.10,1,2,1,first note
1,2,4,4,1.66,1,2,1,first note
2,2,6,2,2.94,1,2,1,first note
3,2,1,2,9.10,2,2,1,second note
4,2,4,4,1.66,2,2,1,second note
5,2,6,2,2.94,2,2,1,second note


In [138]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id\
                AND oi.product_id = oin.product_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.1,1,2,1,first note
1,2,1,2,9.1,2,2,1,second note


# IMPLICIT JOIN SYNTAX 
DON'T USE IT SINCE IF YOU FORGET **WHERE CLAUSE** YOU GET CROSS JOIN SO USE EXPLICIT JOIN INSTEAD.

In [147]:
# THIS IS THE EXPLICIT JOIN, AS BEFORE
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id", con)

,first_name,last_name
0,Ines,Brushfield
1,Ines,Brushfield
2,Clemmie,Betchley
3,Clemmie,Betchley
4,Elka,Twiddell
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Levy,Mynett
9,Levy,Mynett


In [148]:
# THIS IS IMP[LICIT JOIN]
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o, customers c\
            WHERE o.customer_id = c.customer_id", con)

,first_name,last_name
0,Ines,Brushfield
1,Ines,Brushfield
2,Clemmie,Betchley
3,Clemmie,Betchley
4,Elka,Twiddell
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Levy,Mynett
9,Levy,Mynett


In [149]:
# HERE IS WHAT YOU GET IF YOU FORGET WHERE CLAUSE
pd.read_sql("SELECT \
                first_name,\
                last_name\
            FROM orders o, customers c", con)

,first_name,last_name
0,Babara,MacCaffrey
1,Babara,MacCaffrey
2,Babara,MacCaffrey
3,Babara,MacCaffrey
4,Babara,MacCaffrey
...,...,...
95,Levy,Mynett
96,Levy,Mynett
97,Levy,Mynett
98,Levy,Mynett


# OUTER JOINS : LEFT JOIN and RIGHT JOIN

In [165]:
# this is inner join
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,2,Ines,4
1,2,Ines,7
2,5,Clemmie,5
3,5,Clemmie,8
4,6,Elka,1
5,6,Elka,10
6,7,Ilene,2
7,8,Thacher,3
8,10,Levy,6
9,10,Levy,9


In [166]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,1,Babara,NaN
1,2,Ines,4.0
2,2,Ines,7.0
3,3,Freddi,NaN
4,4,Ambur,NaN
5,5,Clemmie,5.0
6,5,Clemmie,8.0
7,6,Elka,1.0
8,6,Elka,10.0
9,7,Ilene,2.0


In [167]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM customers c\
            RIGHT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,2,Ines,4
1,2,Ines,7
2,5,Clemmie,5
3,5,Clemmie,8
4,6,Elka,1
5,6,Elka,10
6,7,Ilene,2
7,8,Thacher,3
8,10,Levy,6
9,10,Levy,9


In [168]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id\
            FROM orders o\
            RIGHT JOIN customers c\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id
0,1,Babara,NaN
1,2,Ines,4.0
2,2,Ines,7.0
3,3,Freddi,NaN
4,4,Ambur,NaN
5,5,Clemmie,5.0
6,5,Clemmie,8.0
7,6,Elka,1.0
8,6,Elka,10.0
9,7,Ilene,2.0


In [158]:
pd.read_sql("SELECT \
                p.product_id,\
                p.name,\
                oi.quantity\
            FROM products p\
            LEFT JOIN order_items oi\
                ON p.product_id = oi.product_id", con)

,product_id,name,quantity
0,1,Foam Dinner Plate,2.0
1,1,Foam Dinner Plate,4.0
2,1,Foam Dinner Plate,10.0
3,2,"Pork - Bacon,back Peameal",3.0
4,2,"Pork - Bacon,back Peameal",4.0
5,3,"Lettuce - Romaine, Heart",10.0
6,3,"Lettuce - Romaine, Heart",7.0
7,3,"Lettuce - Romaine, Heart",4.0
8,3,"Lettuce - Romaine, Heart",7.0
9,4,"Brocolinni - Gaylan, Chinese",4.0


# OUTER JOINS BETWEEN MULTIPLE TABLES

In [4]:
con = connector('sql_store')

In [9]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id,\
                o.shipper_id\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id,shipper_id
0,1,Babara,NaN,NaN
1,2,Ines,7.0,4.0
2,2,Ines,4.0,NaN
3,3,Freddi,NaN,NaN
4,4,Ambur,NaN,NaN
5,5,Clemmie,8.0,NaN
6,5,Clemmie,5.0,3.0
7,6,Elka,10.0,2.0
8,6,Elka,1.0,NaN
9,7,Ilene,2.0,4.0


In [10]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [7]:
pd.read_sql("SELECT *\
            FROM shippers", con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"


In [20]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                o.order_id,\
                s.name AS shipper_name\
            FROM customers c\
            LEFT JOIN orders o\
                ON c.customer_id = o.customer_id\
            LEFT JOIN shippers s\
                ON o.shipper_id = s.shipper_id\
            ORDER BY c.customer_id", con)

,customer_id,first_name,order_id,shipper_name
0,1,Babara,NaN,None
1,2,Ines,4.0,None
2,2,Ines,7.0,"Mraz, Renner and Nolan"
3,3,Freddi,NaN,None
4,4,Ambur,NaN,None
5,5,Clemmie,5.0,Satterfield LLC
6,5,Clemmie,8.0,None
7,6,Elka,1.0,None
8,6,Elka,10.0,Schinner-Predovic
9,7,Ilene,2.0,"Mraz, Renner and Nolan"


In [21]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [22]:
pd.read_sql("SELECT *\
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [23]:
pd.read_sql("SELECT *\
            FROM order_statuses", con)

,order_status_id,name
0,1,Processed
1,2,Shipped
2,3,Delivered


In [24]:
pd.read_sql("SELECT *\
            FROM shippers", con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"


In [36]:
pd.read_sql("SELECT \
                o.order_id,\
                o.order_date,\
                c.first_name,\
                s.name AS shipper,\
                os.name AS status\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id\
            LEFT JOIN shippers s\
                ON o.shipper_id = s.shipper_id\
            JOIN order_statuses os\
                ON o.status = os.order_status_id", con)

,order_id,order_date,first_name,shipper,status
0,8,2018-06-08,Clemmie,None,Processed
1,6,2018-11-18,Levy,None,Processed
2,4,2017-01-22,Ines,None,Processed
3,3,2017-12-01,Thacher,None,Processed
4,1,2019-01-30,Elka,None,Processed
5,10,2018-04-22,Elka,Schinner-Predovic,Shipped
6,9,2017-07-05,Levy,Hettinger LLC,Shipped
7,7,2018-09-22,Ines,"Mraz, Renner and Nolan",Shipped
8,5,2017-08-25,Clemmie,Satterfield LLC,Shipped
9,2,2018-08-02,Ilene,"Mraz, Renner and Nolan",Shipped


# SELF OUTER JOIN

In [38]:
con = connector('sql_hr')

In [40]:
pd.read_sql('SELECT *\
            FROM employees\
            LIMIT 3', con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1


In [43]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name AS employee,\
                m.first_name AS manager\
            FROM employees e\
            JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,employee,manager
0,33391,D'arcy,Yovonnda
1,37851,Sayer,Yovonnda
2,40448,Mindy,Yovonnda
3,56274,Keriann,Yovonnda
4,63196,Alaster,Yovonnda
5,67009,North,Yovonnda
6,67370,Elladine,Yovonnda
7,68249,Nisse,Yovonnda
8,72540,Guthrey,Yovonnda
9,72913,Kass,Yovonnda


In [44]:
pd.read_sql("SELECT \
                e.employee_id,\
                e.first_name AS employee,\
                m.first_name AS manager\
            FROM employees e\
            LEFT JOIN employees m\
                ON e.reports_to = m.employee_id", con)

,employee_id,employee,manager
0,33391,D'arcy,Yovonnda
1,37270,Yovonnda,None
2,37851,Sayer,Yovonnda
3,40448,Mindy,Yovonnda
4,56274,Keriann,Yovonnda
5,63196,Alaster,Yovonnda
6,67009,North,Yovonnda
7,67370,Elladine,Yovonnda
8,68249,Nisse,Yovonnda
9,72540,Guthrey,Yovonnda


# USING CLAUSE

In [45]:
con = connector('sql_store')

In [52]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            JOIN customers c\
                ON o.customer_id = c.customer_id", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


In [53]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


In [56]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name,\
                s.name AS shipper\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)\
            JOIN shippers s\
                USING (shipper_id)", con)

,order_id,first_name,shipper
0,9,Levy,Hettinger LLC
1,10,Elka,Schinner-Predovic
2,5,Clemmie,Satterfield LLC
3,2,Ilene,"Mraz, Renner and Nolan"
4,7,Ines,"Mraz, Renner and Nolan"


In [57]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name,\
                s.name AS shipper\
            FROM orders o\
            JOIN customers c\
                USING (customer_id)\
            LEFT JOIN shippers s\
                USING (shipper_id)", con)

,order_id,first_name,shipper
0,1,Elka,None
1,2,Ilene,"Mraz, Renner and Nolan"
2,3,Thacher,None
3,4,Ines,None
4,5,Clemmie,Satterfield LLC
5,6,Levy,None
6,7,Ines,"Mraz, Renner and Nolan"
7,8,Clemmie,None
8,9,Levy,Hettinger LLC
9,10,Elka,Schinner-Predovic


In [58]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                ON oi.order_id = oin.order_id AND\
                    oi.product_id = oin.product_id", con)

,order_id,product_id,quantity,unit_price,note_id,order_Id,product_id,note
0,2,1,2,9.1,1,2,1,first note
1,2,1,2,9.1,2,2,1,second note


In [59]:
pd.read_sql("SELECT *\
            FROM order_items oi\
            JOIN order_item_notes oin\
                USING (order_id, product_id)", con)

,order_id,product_id,quantity,unit_price,note_id,note
0,2,1,2,9.1,1,first note
1,2,1,2,9.1,2,second note


In [60]:
con = connector('sql_invoicing')

In [64]:
pd.read_sql("SELECT *\
            FROM clients", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [66]:
pd.read_sql("SELECT *\
            FROM payments", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [68]:
pd.read_sql("SELECT *\
            FROM payment_methods", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal
3,4,Wire Transfer


In [73]:
pd.read_sql("SELECT \
            p.date,\
            c.name AS client,\
            p.amount,\
            pm.name AS payment_method\
            FROM payments p\
            JOIN clients c\
                ON p.client_id = c.client_id\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,client,amount,payment_method
0,2019-02-12,Topiclounge,8.18,Credit Card
1,2019-01-03,Vinte,74.55,Credit Card
2,2019-01-11,Yadel,0.03,Credit Card
3,2019-01-26,Topiclounge,87.44,Credit Card
4,2019-01-15,Yadel,80.31,Credit Card
5,2019-01-15,Yadel,68.10,Credit Card
6,2019-01-08,Topiclounge,32.77,Credit Card
7,2019-01-08,Topiclounge,10.00,Cash


In [72]:
pd.read_sql("SELECT \
            p.date,\
            c.name AS client,\
            p.amount,\
            pm.name AS payment_method\
            FROM payments p\
            JOIN clients c\
                USING (client_id)\
            JOIN payment_methods pm\
                ON p.payment_method = pm.payment_method_id", con)

,date,client,amount,payment_method
0,2019-02-12,Topiclounge,8.18,Credit Card
1,2019-01-03,Vinte,74.55,Credit Card
2,2019-01-11,Yadel,0.03,Credit Card
3,2019-01-26,Topiclounge,87.44,Credit Card
4,2019-01-15,Yadel,80.31,Credit Card
5,2019-01-15,Yadel,68.10,Credit Card
6,2019-01-08,Topiclounge,32.77,Credit Card
7,2019-01-08,Topiclounge,10.00,Cash


# NATURAL JOINS -- not recommended due to producing unexpected results

In [78]:
con = connector('sql_store')

In [86]:
pd.read_sql("SELECT \
                o.order_id,\
                c.first_name\
            FROM orders o\
            NATURAL JOIN customers c", con)

,order_id,first_name
0,4,Ines
1,7,Ines
2,5,Clemmie
3,8,Clemmie
4,1,Elka
5,10,Elka
6,2,Ilene
7,3,Thacher
8,6,Levy
9,9,Levy


# CROSS JOIN

In [91]:
pd.read_sql("SELECT \
                c.first_name AS customer,\
                p.name AS product\
            FROM customers c\
            CROSS JOIN products p\
            ORDER BY c.first_name", con)

,customer,product
0,Ambur,Sweet Pea Sprouts
1,Ambur,Island Oasis - Raspberry
2,Ambur,"Lettuce - Romaine, Heart"
3,Ambur,Petit Baguette
4,Ambur,Foam Dinner Plate
...,...,...
95,Thacher,Petit Baguette
96,Thacher,"Pork - Bacon,back Peameal"
97,Thacher,Sweet Pea Sprouts
98,Thacher,Foam Dinner Plate


In [92]:
# the implicit syntax for cross join is as follows (but you use explicit syntax as above for clarity)

pd.read_sql("SELECT \
                c.first_name AS customer,\
                p.name AS product\
            FROM customers c, products p\
            ORDER BY c.first_name", con)

,customer,product
0,Ambur,Sweet Pea Sprouts
1,Ambur,Island Oasis - Raspberry
2,Ambur,"Lettuce - Romaine, Heart"
3,Ambur,Petit Baguette
4,Ambur,Foam Dinner Plate
...,...,...
95,Thacher,Petit Baguette
96,Thacher,"Pork - Bacon,back Peameal"
97,Thacher,Sweet Pea Sprouts
98,Thacher,Foam Dinner Plate


In [95]:
pd.read_sql("SELECT \
                s.name AS shipper,\
                p.name AS product\
            FROM shippers s, products p\
            ORDER BY s.name", con)

,shipper,product
0,Hettinger LLC,Sweet Pea Sprouts
1,Hettinger LLC,"Brocolinni - Gaylan, Chinese"
2,Hettinger LLC,"Lettuce - Romaine, Heart"
3,Hettinger LLC,Sauce - Ranch Dressing
4,Hettinger LLC,Petit Baguette
5,Hettinger LLC,"Pork - Bacon,back Peameal"
6,Hettinger LLC,Island Oasis - Raspberry
7,Hettinger LLC,Foam Dinner Plate
8,Hettinger LLC,Longan
9,Hettinger LLC,Broom - Push


In [96]:
pd.read_sql("SELECT \
                s.name AS shipper,\
                p.name AS product\
            FROM shippers s\
            CROSS JOIN products p\
            ORDER BY s.name", con)

,shipper,product
0,Hettinger LLC,Sweet Pea Sprouts
1,Hettinger LLC,"Brocolinni - Gaylan, Chinese"
2,Hettinger LLC,"Lettuce - Romaine, Heart"
3,Hettinger LLC,Sauce - Ranch Dressing
4,Hettinger LLC,Petit Baguette
5,Hettinger LLC,"Pork - Bacon,back Peameal"
6,Hettinger LLC,Island Oasis - Raspberry
7,Hettinger LLC,Foam Dinner Plate
8,Hettinger LLC,Longan
9,Hettinger LLC,Broom - Push


# UNIONS

In [110]:
pd.read_sql("SELECT \
                order_id,\
                order_date,\
                'Active' AS status\
            FROM orders\
            WHERE order_date >= '2019-01-01'\
            UNION \
            SELECT \
                order_id,\
                order_date,\
                'Archived' AS status\
            FROM orders\
            WHERE order_date <'2019-01-01'\
            ", con)

,order_id,order_date,status
0,1,2019-01-30,Active
1,2,2018-08-02,Archived
2,3,2017-12-01,Archived
3,4,2017-01-22,Archived
4,5,2017-08-25,Archived
5,6,2018-11-18,Archived
6,7,2018-09-22,Archived
7,8,2018-06-08,Archived
8,9,2017-07-05,Archived
9,10,2018-04-22,Archived


In [125]:
pd.read_sql("SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Bronze' AS type\
            FROM customers c\
            WHERE c.points <= 2000\
            UNION\
            SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Silver' AS type\
            FROM customers c\
            WHERE c.points BETWEEN 2000 AND 3000\
            UNION\
            SELECT \
                c.customer_id,\
                c.first_name,\
                c.points,\
                'Gold' AS type\
            FROM customers c\
            WHERE c.points >= 3000\
            ORDER BY first_name", con)

,customer_id,first_name,points,type
0,4,Ambur,457,Bronze
1,1,Babara,2273,Silver
2,5,Clemmie,3675,Gold
3,6,Elka,3073,Gold
4,3,Freddi,2967,Silver
5,7,Ilene,1672,Bronze
6,2,Ines,947,Bronze
7,10,Levy,796,Bronze
8,9,Romola,1486,Bronze
9,8,Thacher,205,Bronze


In [127]:
# no need to give alias to the table name as above so i repeat it in the following: 

pd.read_sql("SELECT \
                customer_id,\
                first_name,\
                points,\
                'Bronze' AS type\
            FROM customers \
            WHERE points <= 2000\
            UNION\
            SELECT \
                customer_id,\
                first_name,\
                points,\
                'Silver' AS type\
            FROM customers \
            WHERE points BETWEEN 2000 AND 3000\
            UNION\
            SELECT \
                customer_id,\
                first_name,\
                points,\
                'Gold' AS type\
            FROM customers \
            WHERE points >= 3000\
            ORDER BY first_name", con)

,customer_id,first_name,points,type
0,4,Ambur,457,Bronze
1,1,Babara,2273,Silver
2,5,Clemmie,3675,Gold
3,6,Elka,3073,Gold
4,3,Freddi,2967,Silver
5,7,Ilene,1672,Bronze
6,2,Ines,947,Bronze
7,10,Levy,796,Bronze
8,9,Romola,1486,Bronze
9,8,Thacher,205,Bronze


# INSERTING - UPDATING - DELETING DATA

In [4]:
con = connector('sql_store')

In [34]:
pd.read_sql("SELECT *\
            FROM customers", con).dtypes

customer_id     int64
first_name     object
last_name      object
birth_date     object
phone          object
address        object
city           object
state          object
points          int64
dtype: object

In [35]:
column_attribute('sql_store', 'customers')

,COLUMN_NAME,DATA_TYPE
0,customer_id,int
1,first_name,varchar
2,last_name,varchar
3,birth_date,date
4,phone,varchar
5,address,varchar
6,city,varchar
7,state,char
8,points,int


# INSERT INTO

pd.read_sql() is used to execute a **SELECT query** and return the results as a pandas DataFrame.

I need to execute an **INSERT query**. Like:

    con = connector('sql_store')
    cursor = con.cursor()

    cursor.execute("""
        INSERT INTO customers (
            first_name,
            last_name,
            birth_date,
            address,
            city,
            state
        ) VALUES (
            'John',
            'Smith',
            '1999-01-04',
            'address',
            'city',
            'CA'
        )
    """)

    con.commit()


In [61]:
pd.read_sql("""
            SELECT *
            FROM customers
            
            """, con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [46]:
cursor = con.cursor()
cursor.execute("INSERT INTO customers\
                VALUES(\
                DEFAULT, \
                'John', \
                'Smith', \
                '1999-01-04', \
                 NULL,\
                'address', \
                'city', \
                'CA', \
                DEFAULT)")
con.commit()

In [53]:
cursor = con.cursor()

cursor.execute("""
    INSERT INTO customers (
        first_name,
        last_name,
        birth_date,
        address,
        city,
        state
    ) VALUES (
        'John',
        'Smith',
        '1999-01-04',
        'address',
        'city',
        'CA'
    )
""")

con.commit()


In [169]:
# to get back the row which was just added

pd.read_sql("SELECT * FROM customers WHERE customer_id = LAST_INSERT_ID()", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,11,John,Smith,1999-01-04,None,address,city,CA,0


In [65]:
pd.read_sql("""

            SELECT *
            FROM shippers
            
            """, con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"


In [67]:
column_attribute('sql_store', 'shippers')

,COLUMN_NAME,DATA_TYPE
0,shipper_id,smallint
1,name,varchar


In [72]:
cursor = con.cursor()

cursor.execute("""
                    
                INSERT INTO shippers (name)
                VALUES  ('shipper1'),
                        ('shipper2'),
                        ('shipper3')

                """)

con.commit()

In [73]:
pd.read_sql("""

            SELECT *
            FROM shippers
            
            """, con)

,shipper_id,name
0,1,Hettinger LLC
1,2,Schinner-Predovic
2,3,Satterfield LLC
3,4,"Mraz, Renner and Nolan"
4,5,"Waters, Mayert and Prohaska"
5,6,shipper1
6,7,shipper2
7,8,shipper3


In [75]:
pd.read_sql("SELECT * FROM shippers WHERE shipper_id = LAST_INSERT_ID()", con)

,shipper_id,name
0,6,shipper1


In [76]:
pd.read_sql("""
            SELECT *
            FROM products

            """, con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [83]:
cursor = con.cursor()

cursor.execute ("""

                INSERT INTO products (name, quantity_in_stock, unit_price)
                VALUES  ('product1', 1, 0),
                        ('product2', 2, 0),
                        ('product3', 3, 0)
            

                """)

con.commit()

In [84]:
pd.read_sql("""

        SELECT *
        FROM products
        
            """, con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [85]:
pd.read_sql("""

SELECT *
FROM orders

""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [87]:
pd.read_sql("""

SELECT *
FROM order_items
LIMIT 4

""", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94


In [89]:
pd.read_sql("""

SELECT *
FROM orders

""", con).columns

Index(['order_id', 'customer_id', 'order_date', 'status', 'comments',
       'shipped_date', 'shipper_id'],
      dtype='object')

In [146]:
default_column_values ('sql_store', 'orders')

Column order_id does not have a default value
Column customer_id does not have a default value
Column order_date does not have a default value
Column status has default value: 1
Column comments has a NULL default value
Column shipped_date has a NULL default value
Column shipper_id has a NULL default value


In [147]:
default_column_values ('sql_store', 'order_items')

Column order_id does not have a default value
Column product_id does not have a default value
Column quantity does not have a default value
Column unit_price does not have a default value


In [161]:
pd.read_sql("""
SELECT *
FROM order_items
""", con).columns

Index(['order_id', 'product_id', 'quantity', 'unit_price'], dtype='object')

In [165]:
pd.read_sql("""
SELECT *
FROM orders
""", con).columns

Index(['order_id', 'customer_id', 'order_date', 'status', 'comments',
       'shipped_date', 'shipper_id'],
      dtype='object')

In [189]:
cursor = con.cursor()

cursor.execute("""

INSERT INTO orders (customer_id, order_date, status)
VALUES (1, '2019-01-02', 1); 

""")

cursor.execute("""

INSERT INTO order_items 
VALUES 
    (LAST_INSERT_ID(), 1, 1, 2)
""")

con.commit()

In [190]:
pd.read_sql("""

SELECT *
FROM orders

""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [191]:
pd.read_sql("""

SELECT *
FROM order_items

""", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


# CREATE TABLE

The table which is created through this technique will not have **auto increment** and **primary key**

In [30]:
con = connector('sql_store')
print(table_names('sql_store'))

         TABLE_NAME
0       EmployeeUNI
1         Employees
2         customers
3  order_item_notes
4       order_items
5    order_statuses
6            orders
7          products
8          shippers


In [31]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [32]:
#con = connector('sql_store')

cursor = con.cursor()

cursor.execute("""

CREATE TABLE orders_archived AS 
SELECT * 
FROM orders

""")

con.commit() 

In [33]:
print(table_names('sql_store'))

         TABLE_NAME
0       EmployeeUNI
1         Employees
2         customers
3  order_item_notes
4       order_items
5    order_statuses
6            orders
7   orders_archived
8          products
9          shippers


In [34]:
primary_key_finder('sql_store')

customers customer_id PRIMARY
order_item_notes note_id PRIMARY
order_items order_id PRIMARY
order_items product_id PRIMARY
order_statuses order_status_id PRIMARY
orders order_id PRIMARY
products product_id PRIMARY
shippers shipper_id PRIMARY


In [36]:
con = connector('sql_store')
pd.read_sql("""
SELECT *
FROM orders_archived

""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [37]:
con = connector('sql_store')

cursor = con.cursor()

cursor.execute("""
TRUNCATE TABLE orders_archived
""")

# commit the changes to the database
con.commit()

# close the database connection
con.close()



In [38]:
con = connector('sql_store')
pd.read_sql("""
SELECT *
FROM orders_archived

""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id


In [39]:
cursor = con.cursor()

cursor.execute("""

INSERT INTO orders_archived 
SELECT *
FROM orders
WHERE order_date <'2019-01-01'

""")

con.commit()

In [40]:
con = connector('sql_store')

pd.read_sql("""
SELECT *
FROM orders_archived

""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,2,7,2018-08-02,2,None,2018-08-03,4.0
1,3,8,2017-12-01,1,None,None,NaN
2,4,2,2017-01-22,1,None,None,NaN
3,5,5,2017-08-25,2,,2017-08-26,3.0
4,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
5,7,2,2018-09-22,2,None,2018-09-23,4.0
6,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
7,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
8,10,6,2018-04-22,2,None,2018-04-23,2.0


In [42]:
con = connector('sql_invoicing')

In [53]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [56]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [58]:
pd.read_sql("""
SELECT *
FROM invoices
JOIN clients
    USING (client_id)
WHERE payment_date IS NOT NULL
""", con)

,client_id,invoice_id,number,invoice_total,payment_total,invoice_date,due_date,payment_date,name,address,city,state,phone
0,1,6,75-587-6626,157.78,74.55,2019-01-29,2019-02-18,2019-01-03,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,3,11,20-848-0181,126.15,0.03,2019-01-07,2019-01-27,2019-01-11,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
2,3,15,55-105-9605,167.29,80.31,2019-11-25,2019-12-15,2019-01-15,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,3,17,33-615-4694,126.38,68.10,2019-07-30,2019-08-19,2019-01-15,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
4,5,2,03-898-6735,175.32,8.18,2019-06-11,2019-07-01,2019-02-12,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129
5,5,13,41-666-1035,135.01,87.44,2019-06-25,2019-07-15,2019-01-26,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129
6,5,18,52-269-9803,180.17,42.77,2019-05-23,2019-06-12,2019-01-08,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [59]:
print(table_names('sql_invoicing'))

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [60]:
cursor = con.cursor()

cursor.execute("""

CREATE TABLE invoices_archived AS
SELECT 
    i.invoice_id,
    i.number,
    c.name AS client,
    i.invoice_total,
    i.payment_total,
    i.payment_date,
    i.due_date
FROM invoices i
JOIN clients c
    USING (client_id)
WHERE payment_date IS NOT NULL

""")

con.commit()


In [61]:
print(table_names('sql_invoicing'))

          TABLE_NAME
0            clients
1           invoices
2  invoices_archived
3    payment_methods
4           payments


In [63]:
pd.read_sql("""
SELECT *
FROM invoices_archived
""", con)

,invoice_id,number,client,invoice_total,payment_total,payment_date,due_date
0,6,75-587-6626,Vinte,157.78,74.55,2019-01-03,2019-02-18
1,11,20-848-0181,Yadel,126.15,0.03,2019-01-11,2019-01-27
2,15,55-105-9605,Yadel,167.29,80.31,2019-01-15,2019-12-15
3,17,33-615-4694,Yadel,126.38,68.10,2019-01-15,2019-08-19
4,2,03-898-6735,Topiclounge,175.32,8.18,2019-02-12,2019-07-01
5,13,41-666-1035,Topiclounge,135.01,87.44,2019-01-26,2019-07-15
6,18,52-269-9803,Topiclounge,180.17,42.77,2019-01-08,2019-06-12


# UPDATE and SET

In [45]:
con = connector('sql_invoicing')

In [35]:
print(table_names('sql_invoicing'))

          TABLE_NAME
0            clients
1           invoices
2  invoices_archived
3    payment_methods
4           payments


In [59]:
pd.read_sql("""

SELECT *
FROM clients
LIMIT 4
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784


In [60]:
con = connector('sql_invoicing')
pd.read_sql("""

SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [61]:
cursor = con.cursor()

cursor.execute("""

UPDATE invoices
SET payment_total = 10, payment_date = '2022-01-01'
WHERE invoice_id  = 3


""")

con.commit()

cursor.close()
con.close()

In [62]:
con = connector('sql_invoicing')

In [63]:
pd.read_sql("""
SELECT *
FROM invoices

""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,10.00,2019-07-31,2019-08-20,2022-01-01
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [64]:
cursor = con.cursor()

cursor.execute("""

UPDATE invoices
SET payment_total = DEFAULT, payment_date = NULL
WHERE invoice_id = 3
""")

con.commit()


In [65]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [69]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices
SET 
    payment_total = invoice_total * 0.5,
    payment_date = due_date
WHERE invoice_id = 3
""")

con.commit()


In [70]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [71]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices
SET 
    payment_total = invoice_total * 0.5,
    payment_date = due_date
WHERE client_id = 3

""")

con.commit()


In [72]:
pd.read_sql("""
SELECT *
FROM invoices

""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,76.11,2019-03-08,2019-03-28,2019-03-28
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,66.94,2019-09-04,2019-09-24,2019-09-24
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [73]:
con = connector('sql_store')

In [75]:
pd.read_sql("""
SELECT *
FROM customers
LIMIT 3
""", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [77]:
cursor = con.cursor()

cursor.execute("""
UPDATE customers
SET points = points + 50
WHERE birth_date < '1990-01-01'

""")

con.commit()



In [78]:
pd.read_sql("""
SELECT *
FROM customers
""", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2323
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,997
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,3017
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,507
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3725
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1722
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,846


In [87]:
con = connector('sql_invoicing')

In [88]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [89]:
pd.read_sql("""
SELECT client_id
FROM clients
WHERE name = 'Topiclounge'
""", con)

,client_id
0,5


In [90]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,76.11,2019-03-08,2019-03-28,2019-03-28
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,66.94,2019-09-04,2019-09-24,2019-09-24
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [91]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices
SET 
    payment_total = invoice_total * 0.5,
    payment_date = due_date
WHERE client_id =
                (SELECT client_id
                FROM clients
                WHERE name = 'Topiclounge')
""")

con.commit()


In [92]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,87.66,2019-06-11,2019-07-01,2019-07-01
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,76.11,2019-03-08,2019-03-28,2019-03-28
4,5,87-052-3121,5,169.36,84.68,2019-07-18,2019-08-07,2019-08-07
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,66.94,2019-09-04,2019-09-24,2019-09-24
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,86.09,2019-07-09,2019-07-29,2019-07-29
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [97]:
pd.read_sql("""
SELECT *
FROM clients""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [98]:
pd.read_sql("""
SELECT client_id
FROM clients
WHERE state IN ('CA', 'NY')
""", con)

,client_id
0,1
1,3


In [99]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices
SET
    payment_total = invoice_total * 0.5,
    payment_date = due_date
WHERE client_id IN
            (SELECT client_id
            FROM clients
            WHERE state IN ('CA', 'NY'))

""")

con.commit()


In [100]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,87.66,2019-06-11,2019-07-01,2019-07-01
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,76.11,2019-03-08,2019-03-28,2019-03-28
4,5,87-052-3121,5,169.36,84.68,2019-07-18,2019-08-07,2019-08-07
5,6,75-587-6626,1,157.78,78.89,2019-01-29,2019-02-18,2019-02-18
6,7,68-093-9863,3,133.87,66.94,2019-09-04,2019-09-24,2019-09-24
7,8,78-145-1093,1,189.12,94.56,2019-05-20,2019-06-09,2019-06-09
8,9,77-593-0081,5,172.17,86.09,2019-07-09,2019-07-29,2019-07-29
9,10,48-266-1517,1,159.50,79.75,2019-06-30,2019-07-20,2019-07-20


In [101]:
con = connector('sql_store')

In [106]:
pd.read_sql("""
SELECT *
FROM orders
LIMIT 2
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0


In [108]:
pd.read_sql("""
SELECT *
FROM customers
WHERE points > 3000

""", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,3017
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3725
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [109]:
pd.read_sql("""
SELECT customer_id
FROM customers
WHERE points > 3000

""", con)

,customer_id
0,3
1,5
2,6


In [110]:
cursor = con.cursor()

cursor.execute("""
UPDATE orders
SET comments = 'GOLDEN'
WHERE customer_id IN 
                (SELECT customer_id
                FROM customers
                WHERE points > 3000)
""")

con.commit()

In [111]:
pd.read_sql("""
SELECT *
FROM orders
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,GOLDEN,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,GOLDEN,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,GOLDEN,None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,GOLDEN,2018-04-23,2.0


# DELETE

In [5]:
con = connector('sql_invoicing')

In [6]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,87.66,2019-06-11,2019-07-01,2019-07-01
2,3,20-228-0335,5,147.99,74.00,2019-07-31,2019-08-20,2019-08-20
3,4,56-934-0748,3,152.21,76.11,2019-03-08,2019-03-28,2019-03-28
4,5,87-052-3121,5,169.36,84.68,2019-07-18,2019-08-07,2019-08-07
5,6,75-587-6626,1,157.78,78.89,2019-01-29,2019-02-18,2019-02-18
6,7,68-093-9863,3,133.87,66.94,2019-09-04,2019-09-24,2019-09-24
7,8,78-145-1093,1,189.12,94.56,2019-05-20,2019-06-09,2019-06-09
8,9,77-593-0081,5,172.17,86.09,2019-07-09,2019-07-29,2019-07-29
9,10,48-266-1517,1,159.50,79.75,2019-06-30,2019-07-20,2019-07-20


In [7]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [10]:
pd.read_sql("""
SELECT client_id
FROM clients
WHERE NAME = 'Myworks'
""", con)

,client_id
0,2


In [ ]:
cursor = con.cursor()

cursor.execute("""
DELETE FROM invoices
WHERE client_id = 
                (SELECT client_id
                FROM clients
                WHERE name = 'Myworks')

""")
con.commit()

In [6]:
# I restored the databases, in MySQL Workbench, to clear evrything and got back to the initial stage

In [4]:
con = connector('sql_invoicing')

In [5]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


# Summarizing Data 

# Aggregate functions
+ MAX() 
+ MIN() 
+ COUNT() / COUNT(*) 
+ AVG() 
+ SUM()

In [10]:
con = connector('sql_invoicing')

In [21]:
pd.read_sql("""
SELECT MAX(invoice_total)
FROM invoices
""", con)

,MAX(invoice_total)
0,189.12


In [23]:
pd.read_sql("""
SELECT MAX(invoice_total) AS highest
FROM invoices
""", con)

,highest
0,189.12


In [27]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total
FROM invoices
""", con)

,highest,lowest,average,total
0,189.12,101.79,152.388235,2590.6


In [28]:
pd.read_sql("""
SELECT
    MAX(payment_date) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total
FROM invoices
""", con)

,highest,lowest,average,total
0,2019-02-12,101.79,152.388235,2590.6


In [29]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    COUNT(invoice_total) AS number_of_invoices
FROM invoices
""", con)

,highest,lowest,average,total,number_of_invoices
0,189.12,101.79,152.388235,2590.6,17


In [30]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    COUNT(invoice_total) AS number_of_invoices,
    COUNT(payment_date) AS count_of_payments
FROM invoices
""", con)

,highest,lowest,average,total,number_of_invoices,count_of_payments
0,189.12,101.79,152.388235,2590.6,17,7


In [33]:
pd.read_sql("""
SELECT payment_date
FROM invoices""", con)

,payment_date
0,None
1,2019-02-12
2,None
3,None
4,None
5,2019-01-03
6,None
7,None
8,None
9,None


In [34]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    COUNT(invoice_total) AS number_of_invoices,
    COUNT(payment_date) AS count_of_payments,
    COUNT(*) AS total_records
FROM invoices
""", con)

,highest,lowest,average,total,number_of_invoices,count_of_payments,total_records
0,189.12,101.79,152.388235,2590.6,17,7,17


In [36]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    SUM(invoice_total *1.1) AS total_10_percent_increment,
    COUNT(*) AS total_records
FROM invoices
""", con)

,highest,lowest,average,total,total_10_percent_increment,total_records
0,189.12,101.79,152.388235,2590.6,2849.66,17


In [37]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    SUM(invoice_total *1.1) AS total_10_percent_increment,
    COUNT(*) AS total_records
FROM invoices
WHERE invoice_date > '2019-07-01'
""", con)

,highest,lowest,average,total,total_10_percent_increment,total_records
0,172.17,126.38,150.218571,1051.53,1156.683,7


In [40]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    SUM(invoice_total *1.1) AS total_10_percent_increment,
    COUNT(*) AS total_records,
    COUNT(client_id) AS number_of_clieants
FROM invoices
WHERE invoice_date > '2019-07-01'
""", con)

,highest,lowest,average,total,total_10_percent_increment,total_records,number_of_clieants
0,172.17,126.38,150.218571,1051.53,1156.683,7,7


In [41]:
pd.read_sql("""
SELECT
    MAX(invoice_total) AS highest,
    MIN(invoice_total) AS lowest,
    AVG(invoice_total) AS average, 
    SUM(invoice_total) AS total,
    SUM(invoice_total *1.1) AS total_10_percent_increment,
    COUNT(*) AS total_records,
    COUNT(DISTINCT client_id) AS number_of_clieants
FROM invoices
WHERE invoice_date > '2019-07-01'
""", con)

,highest,lowest,average,total,total_10_percent_increment,total_records,number_of_clieants
0,172.17,126.38,150.218571,1051.53,1156.683,7,3


In [43]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12


In [80]:
pd.read_sql("""
SELECT 
    'First half of 2019' AS date_range,
    SUM(invoice_total) AS total_sales, 
    SUM(payment_total) AS total_payments, 
    SUM(invoice_total - payment_total) AS 'what_we_expect' 
FROM invoices
WHERE invoice_date 
    BETWEEN'2019-01-01' AND '2019-06-30'

UNION
SELECT 
    'Second half of 2019' AS date_range,
    SUM(invoice_total) AS total_sales, 
    SUM(payment_total) AS total_payments,
    SUM(invoice_total - payment_total) AS 'what_we_expect' 
FROM invoices
WHERE invoice_date 
    BETWEEN '2019-07-01' AND '2019-12-31'

UNION
SELECT 
    'Total' AS date_range, 
    SUM(invoice_total) AS total_sales, 
    SUM(payment_total) AS total_payments,
    SUM(invoice_total - payment_total) AS 'what_we_expect' 
FROM invoices
WHERE invoice_date 
    BETWEEN '2019-01-01' AND '2019-12-31'
""", con)

,date_range,total_sales,total_payments,what_we_expect
0,First half of 2019,1539.07,212.97,1326.10
1,Second half of 2019,1051.53,148.41,903.12
2,Total,2590.60,361.38,2229.22


In [83]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 4
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None


In [86]:
pd.read_sql("""
SELECT COUNT(payment_date)
FROM invoices
WHERE payment_date IS NOT NULL
""", con)

,COUNT(payment_date)
0,7


In [87]:
pd.read_sql("""
SELECT COUNT(*)
FROM invoices
""", con)

,COUNT(*)
0,17


# GROUP BY

In [93]:
pd.read_sql("""
SELECT *
FROM clients
LIMIT 4""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784


In [92]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 4""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None


In [125]:
pd.read_sql("""
SELECT
    client_id,
    SUM(invoice_total) AS total_sales
FROM invoices
GROUP BY client_id
""", con)

,client_id,total_sales
0,1,802.89
1,2,101.79
2,3,705.90
3,5,980.02


In [126]:
pd.read_sql("""
SELECT
    client_id,
    SUM(invoice_total) AS total_sales
FROM invoices
GROUP BY client_id
ORDER BY total_sales DESC
""", con)

,client_id,total_sales
0,5,980.02
1,1,802.89
2,3,705.90
3,2,101.79


In [136]:
pd.read_sql("""
SELECT
    client_id,
    SUM(invoice_total) AS total_sales
FROM invoices
WHERE invoice_date >= '2019-07-01'
GROUP BY client_id
ORDER BY total_sales DESC
""", con)

,client_id,total_sales
0,5,489.52
1,3,427.54
2,1,134.47


In [159]:
pd.read_sql("""
SELECT
    c.name, 
    c.client_id,
    SUM(i.invoice_total) AS total_sales
FROM invoices i
JOIN clients c
    USING(client_id)
GROUP BY client_id
ORDER BY total_sales DESC
""", con)

,name,client_id,total_sales
0,Topiclounge,5,980.02
1,Vinte,1,802.89
2,Yadel,3,705.90
3,Myworks,2,101.79


In [170]:
# per state and city

pd.read_sql("""
SELECT
    c.state,
    c.city, 
    SUM(i.invoice_total) AS total_sales
FROM invoices i
JOIN clients c
    USING(client_id)
GROUP BY c.state, c.city
""", con)

,state,city,total_sales
0,WV,Huntington,101.79
1,OR,Portland,980.02
2,CA,San Francisco,705.90
3,NY,Syracuse,802.89


In [172]:
pd.read_sql("""
SELECT *
FROM payments
LIMIT 3
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1


In [176]:
print (table_names('sql_invoicing'))

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [177]:
pd.read_sql("""
SELECT *
FROM payment_methods
LIMIT 3
""", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal


In [255]:
pd.read_sql("""
SELECT 
    date,
    SUM(amount) AS total_payments, 
    pm.name AS payment_method 
FROM payments p
JOIN payment_methods pm
    ON p.payment_method = pm.payment_method_id
GROUP BY date, payment_method
ORDER BY date
""", con)

,date,total_payments,payment_method
0,2019-01-03,74.55,Credit Card
1,2019-01-08,32.77,Credit Card
2,2019-01-08,10.00,Cash
3,2019-01-11,0.03,Credit Card
4,2019-01-15,148.41,Credit Card
5,2019-01-26,87.44,Credit Card
6,2019-02-12,8.18,Credit Card


# HAVING

In [260]:
pd.read_sql("""
SELECT 
    client_id,
    SUM(invoice_total) AS total_sales
FROM invoices
GROUP BY client_id
HAVING total_sales > 200
""", con)

,client_id,total_sales
0,1,802.89
1,3,705.90
2,5,980.02


In [263]:
pd.read_sql("""
SELECT 
    client_id,
    SUM(invoice_total) AS total_sales,
    COUNT(*) AS number_of_invoices
FROM invoices
GROUP BY client_id
HAVING total_sales > 200 AND number_of_invoices > 5
""", con)

,client_id,total_sales,number_of_invoices
0,5,980.02,6


In [264]:
con = connector('sql_store')

In [265]:
print(table_names('sql_store'))

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [280]:
pd.read_sql("""
SELECT *
FROM customers

""", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [296]:
pd.read_sql("""
SELECT *
FROM order_items
LIMIT 2
""", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10


In [271]:
pd.read_sql("""
SELECT *
FROM orders
LIMIT 2
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0


In [333]:
pd.read_sql("""
SELECT   
    c.customer_id, 
    c.first_name,
    c.last_name,
    SUM(oi.quantity * oi.unit_price) AS total_spending
FROM customers c
JOIN orders o
    USING (customer_id)
JOIN order_items oi
    USING (order_id)
WHERE state = 'VA'
GROUP BY 
    c.customer_id, 
    c.first_name, 
    c.last_name
HAVING total_spending > 100
""", con)

,customer_id,first_name,last_name,total_spending
0,2,Ines,Brushfield,157.92


# WITH ROLLUP

In [336]:
con = connector('sql_invoicing')

In [343]:
pd.read_sql("""
SELECT 
    client_id,
    SUM(invoice_total) AS total_sales
FROM invoices
GROUP BY client_id WITH ROLLUP
""", con)

,client_id,total_sales
0,1.0,802.89
1,2.0,101.79
2,3.0,705.90
3,5.0,980.02
4,NaN,2590.60


In [352]:
pd.read_sql("""
SELECT 
    state, 
    city, 
    SUM(invoice_total) AS total_sales
FROM invoices
JOIN clients c USING (client_id)
GROUP BY state, city WITH ROLLUP
""", con)

,state,city,total_sales
0,CA,San Francisco,705.90
1,CA,None,705.90
2,NY,Syracuse,802.89
3,NY,None,802.89
4,OR,Portland,980.02
5,OR,None,980.02
6,WV,Huntington,101.79
7,WV,None,101.79
8,None,None,2590.60


In [353]:
pd.read_sql("""
SELECT 
    state, 
    city, 
    SUM(invoice_total) AS total_sales
FROM invoices
JOIN clients c USING (client_id)
GROUP BY state, city WITH ROLLUP
""", con)

,state,city,total_sales
0,CA,San Francisco,705.90
1,CA,None,705.90
2,NY,Syracuse,802.89
3,NY,None,802.89
4,OR,Portland,980.02
5,OR,None,980.02
6,WV,Huntington,101.79
7,WV,None,101.79
8,None,None,2590.60


In [355]:
print(table_names('sql_invoicing'))

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [357]:
pd.read_sql("""
SELECT *
FROM payment_methods
""", con)

,payment_method_id,name
0,1,Credit Card
1,2,Cash
2,3,PayPal
3,4,Wire Transfer


In [364]:
pd.read_sql("""
SELECT 
    pm.name AS payment_method,
    SUM(amount) AS total
FROM payments p
JOIN payment_methods pm
    ON p.payment_method = pm.payment_method_id
GROUP BY pm.name WITH ROLLUP 
""", con)

# when we use a ROLLUP operator we cannot use a column alias in the GROUP BY clause so the following won't work:
# GROUP BY payment_method WITH ROLLUP

,payment_method,total
0,Cash,10.00
1,Credit Card,351.38
2,None,361.38


# Writing Complex Query

# Subqueries

In [11]:
con = connector('sql_store')

In [14]:
pd.read_sql("""
SELECT *
FROM products
WHERE unit_price > (
            SELECT unit_price
            FROM products
            WHERE product_id=3)
""", con)

,product_id,name,quantity_in_stock,unit_price
0,2,"Pork - Bacon,back Peameal",49,4.65
1,4,"Brocolinni - Gaylan, Chinese",90,4.53


In [15]:
con = connector('sql_hr')

In [70]:
table_names('sql_hr')

,TABLE_NAME
0,employees
1,offices


In [69]:
pd.read_sql("""
SELECT *
FROM employees
WHERE salary > (
            SELECT
                AVG(salary)
            FROM employees
            )

""", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,37851,Sayer,Matterson,Statistician III,98926,37270,1
1,40448,Mindy,Crissil,Staff Scientist,94860,37270,1
2,56274,Keriann,Alloisi,VP Marketing,110150,37270,1
3,67009,North,de Clerc,VP Product Management,114257,37270,2
4,67370,Elladine,Rising,Social Worker,96767,37270,2
5,72540,Guthrey,Iacopetti,Office Assistant I,117690,37270,3
6,72913,Kass,Hefferan,Computer Systems Analyst IV,96401,37270,3
7,76196,Mirilla,Janowski,Cost Accountant,119241,37270,3
8,84791,Hazel,Tarbert,General Manager,93760,37270,4
9,95213,Cole,Kesterton,Pharmacist,86119,37270,4


# IN

In [72]:
con = connector('sql_store')

In [78]:
print(table_names('sql_store'))

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [98]:
pd.read_sql("""
SELECT DISTINCT product_id
FROM order_items
 
""", con)

,product_id
0,1
1,2
2,3
3,4
4,5
5,6
6,8
7,9
8,10


In [101]:
pd.read_sql("""
SELECT *
FROM products
WHERE product_id NOT IN (
        SELECT DISTINCT product_id
        FROM order_items
        )
""", con)

,product_id,name,quantity_in_stock,unit_price
0,7,Sweet Pea Sprouts,98,3.29


In [102]:
con = connector('sql_invoicing')

In [104]:
print(table_names('sql_invoicing'))

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [110]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [111]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [113]:
pd.read_sql("""
SELECT *
FROM clients
WHERE client_id NOT IN(
        SELECT DISTINCT client_id
        FROM invoices
)
""", con)

,client_id,name,address,city,state,phone
0,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784


# Subqueries vs. JOINs

readability and performance determines which way to go! Later, we talk about execution plans where we learn how to write a query that executes faster. But here, let's assume both queries (i.e., the one using subqueries as above and the one using JOIN as below executes similarly in terms of speed) then we pay attention to the readability. 

In this particular example using subqueries makes our query more readable, which is not always true, and sometimes using subqueries makes queries complicated and it is better to go with JOIN! So always pay great attention to the readability of your code. 

In [120]:
pd.read_sql("""
SELECT *
FROM clients
LEFT JOIN invoices USING (client_id)
WHERE invoice_id IS NULL
""", con)

,client_id,name,address,city,state,phone,invoice_id,number,invoice_total,payment_total,invoice_date,due_date,payment_date
0,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784,None,None,None,None,None,None,None


In [10]:
con = connector('sql_store')

In [21]:
table_names('sql_store')

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [134]:
pd.read_sql("""
SELECT *
FROM products
LIMIT 4
""", con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53


In [133]:
pd.read_sql("""
SELECT *
FROM order_items
LIMIT 2
""", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10


In [25]:
pd.read_sql("""
SELECT *
FROM customers

""", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [143]:
pd.read_sql("""
SELECT *
FROM orders
LIMIT 2
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0


In [29]:
# solving using JOIN 

pd.read_sql("""
SELECT 
    DISTINCT customer_id,
    first_name,
    last_name
FROM customers
JOIN orders
    USING (customer_id)
JOIN order_items
    USING (order_id)
WHERE product_id = 3
""", con)

,customer_id,first_name,last_name
0,8,Thacher,Naseby
1,2,Ines,Brushfield
2,10,Levy,Mynett


In [24]:
# Solving the problem using subquery - my solution 

pd.read_sql("""
SELECT
    customer_id,
    first_name,
    last_name
FROM customers
WHERE customer_id IN (
    SELECT customer_id
    FROM orders
    WHERE order_id IN (
            SELECT order_id
            FROM order_items
            WHERE product_id = 3
            )
)
""", con)

,customer_id,first_name,last_name
0,2,Ines,Brushfield
1,8,Thacher,Naseby
2,10,Levy,Mynett


In [31]:
# Mosh approach to solve the problem using subquery 

pd.read_sql("""
SELECT
    customer_id,
    first_name,
    last_name
FROM customers
WHERE customer_id IN (
    SELECT customer_id
    FROM orders
    JOIN order_items
        USING (order_id)
    WHERE product_id = 3
            )

""", con)

,customer_id,first_name,last_name
0,8,Thacher,Naseby
1,2,Ines,Brushfield
2,10,Levy,Mynett


In this example, using JOIN makes our query more readable compared to using subquery! 

# ALL

In [33]:
con = connector('sql_invoicing')

In [37]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 4
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None


Sometimes your subquery returns a single value, sometimes it returns a list, sometimes returns a table:

In [36]:
# Here my subquery returns a single value:

In [42]:
pd.read_sql("""
SELECT *
FROM invoices
WHERE invoice_total > (
            SELECT MAX(invoice_total)
            FROM invoices
            WHERE client_id = 3
            )
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
1,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
2,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
3,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
4,18,52-269-9803,5,180.17,42.77,2019-05-23,2019-06-12,2019-01-08


In [43]:
# Here the subquery returns a list:

In [45]:
pd.read_sql("""
SELECT *
FROM invoices
WHERE invoice_total > ALL (
            SELECT invoice_total
            FROM invoices
            WHERE client_id = 3

            )
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
1,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
2,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
3,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
4,18,52-269-9803,5,180.17,42.77,2019-05-23,2019-06-12,2019-01-08


Both queries above (the one using MAX() and the one using ALL) are readable, so go with the one you like more!

# ANY or SOME (they are equivalent)

In [55]:
pd.read_sql("""
SELECT *
FROM clients
WHERE client_id IN (
        SELECT client_id
        FROM invoices
        GROUP BY client_id
        HAVING COUNT(*) >=2
)

""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
2,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [56]:
# another way to write the above query

In [60]:
pd.read_sql("""
SELECT *
FROM clients
WHERE client_id = ANY (
        SELECT client_id
        FROM invoices
        GROUP BY client_id
        HAVING COUNT(*) >=2
)

""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
2,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


(= ANY) is equivalent to (IN) (choose what you prefer, it is up to you)

# Correlated Subqueries

In correlated subqueries we have a correlation with the outer query, like we are referencing the alias from the outer query. 

In [61]:
con = connector('sql_hr')

In [62]:
table_names('sql_hr')

  TABLE_NAME
0  employees
1    offices


In [64]:
pd.read_sql("""
SELECT *
FROM employees
LIMIT 3
""", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,33391,D'arcy,Nortunen,Account Executive,62871,37270.0,1
1,37270,Yovonnda,Magrannell,Executive Secretary,63996,NaN,10
2,37851,Sayer,Matterson,Statistician III,98926,37270.0,1


In [66]:
pd.read_sql("""
SELECT *
FROM offices
LIMIT 3
""", con)

,office_id,address,city,state
0,1,03 Reinke Trail,Cincinnati,OH
1,2,5507 Becker Terrace,New York City,NY
2,3,54 Northland Court,Richmond,VA


In [74]:
pd.read_sql("""
SELECT *
FROM employees e
WHERE salary > (
            SELECT AVG(salary)
            FROM employees
            WHERE office_id = e.office_id
            )
""", con)

,employee_id,first_name,last_name,job_title,salary,reports_to,office_id
0,37851,Sayer,Matterson,Statistician III,98926,37270,1
1,40448,Mindy,Crissil,Staff Scientist,94860,37270,1
2,56274,Keriann,Alloisi,VP Marketing,110150,37270,1
3,67009,North,de Clerc,VP Product Management,114257,37270,2
4,67370,Elladine,Rising,Social Worker,96767,37270,2
5,72540,Guthrey,Iacopetti,Office Assistant I,117690,37270,3
6,76196,Mirilla,Janowski,Cost Accountant,119241,37270,3
7,84791,Hazel,Tarbert,General Manager,93760,37270,4
8,95213,Cole,Kesterton,Pharmacist,86119,37270,4
9,98374,Estrellita,Daleman,Staff Accountant IV,70187,37270,5


In [76]:
con = connector('sql_invoicing')

In [77]:
table_names('sql_invoicing')

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [79]:
pd.read_sql("""
SELECT *
FROM clients
LIMIT 2
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170


In [82]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 3
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None


In [86]:
pd.read_sql("""
SELECT *
FROM invoices i
WHERE invoice_total > (
            SELECT AVG(invoice_total)
            FROM invoices
            WHERE client_id = i.client_id
            )
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
1,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
2,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
3,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
4,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
5,15,55-105-9605,3,167.29,80.31,2019-11-25,2019-12-15,2019-01-15
6,16,10-451-8824,1,162.02,0.00,2019-03-30,2019-04-19,None
7,18,52-269-9803,5,180.17,42.77,2019-05-23,2019-06-12,2019-01-08


# EXISTS

In [98]:
pd.read_sql("""
SELECT *
FROM clients
JOIN invoices
    USING (client_id)

""", con)

,client_id,name,address,city,state,phone,invoice_id,number,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305,6,75-587-6626,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
1,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305,8,78-145-1093,189.12,0.00,2019-05-20,2019-06-09,None
2,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305,10,48-266-1517,159.50,0.00,2019-06-30,2019-07-20,None
3,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305,16,10-451-8824,162.02,0.00,2019-03-30,2019-04-19,None
4,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305,19,83-559-4105,134.47,0.00,2019-11-23,2019-12-13,None
5,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170,1,91-953-3396,101.79,0.00,2019-03-09,2019-03-29,None
6,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037,4,56-934-0748,152.21,0.00,2019-03-08,2019-03-28,None
7,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037,7,68-093-9863,133.87,0.00,2019-09-04,2019-09-24,None
8,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037,11,20-848-0181,126.15,0.03,2019-01-07,2019-01-27,2019-01-11
9,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037,15,55-105-9605,167.29,80.31,2019-11-25,2019-12-15,2019-01-15


In [95]:
pd.read_sql("""
SELECT *
FROM clients
WHERE client_id IN(
        SELECT DISTINCT client_id
        FROM invoices
        )
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [102]:
pd.read_sql("""
SELECT *
FROM clients c
WHERE EXISTS (
        SELECT DISTINCT client_id
        FROM invoices
        WHERE client_id = c.client_id
        )
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


### Takeaway:

When we use IN operator MySQL executes the subquery and returns the results to the WHERE clause, which in our case is a list of 4 client_id. What if I have millions of elements in this list? negative effect on the performance and in these situations I need to use EXISTS operator, where the subquery does not return a result to the outer query and instead it returns an indication of whether any rows in the subquery matches the serach condition! This enhances performance.

So if the subquery we write after the IN operator produces a large result set, it is more efficient to use EXISTS operator.

In [103]:
con = connector('sql_store')

In [104]:
table_names('sql_store')

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [107]:
pd.read_sql("""
SELECT *
FROM orders
LIMIT 2
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0


In [113]:
pd.read_sql("""
SELECT products
SELECT DISTINCT product_id
FROM order_items

""", con)

,product_id
0,1
1,2
2,3
3,4
4,5
5,6
6,8
7,9
8,10


In [122]:
pd.read_sql("""
SELECT *
FROM products p
WHERE NOT EXISTS (
        SELECT DISTINCT product_id
        FROM order_items
        WHERE product_id = p.product_id
        )
""", con)

,product_id,name,quantity_in_stock,unit_price
0,7,Sweet Pea Sprouts,98,3.29


In [124]:
# using IN operator 
pd.read_sql("""
SELECT *
FROM products 
WHERE product_id NOT IN (
        SELECT DISTINCT product_id
        FROM order_items
        )
""", con)

,product_id,name,quantity_in_stock,unit_price
0,7,Sweet Pea Sprouts,98,3.29


# Subqueries in the SELECT clause

So far we only used subqueries in the WHERE clause of a SELECT clause, we can also use subqueries in the SELECT clase and also FROM clause

In [126]:
con = connector('sql_invoicing')

In [135]:
pd.read_sql("""
SELECT 
    invoice_id,
    invoice_total,
    (
        SELECT AVG(invoice_total)
        FROM invoices 
    ) AS invoice_average,
    invoice_total - (
        SELECT AVG(invoice_total)
        FROM invoices 
    ) AS invoice_dif
FROM invoices
""", con)

,invoice_id,invoice_total,invoice_average,invoice_dif
0,1,101.79,152.388235,-50.598235
1,2,175.32,152.388235,22.931765
2,3,147.99,152.388235,-4.398235
3,4,152.21,152.388235,-0.178235
4,5,169.36,152.388235,16.971765
5,6,157.78,152.388235,5.391765
6,7,133.87,152.388235,-18.518235
7,8,189.12,152.388235,36.731765
8,9,172.17,152.388235,19.781765
9,10,159.50,152.388235,7.111765


In [136]:
# above query is repeatative and better idea is as follows

In [140]:
pd.read_sql("""
SELECT 
    invoice_id,
    invoice_total,
    (
        SELECT AVG(invoice_total)
        FROM invoices 
    ) AS invoice_average,
    invoice_total - (SELECT invoice_average) AS invoice_dif
FROM invoices
""", con)

,invoice_id,invoice_total,invoice_average,invoice_dif
0,1,101.79,152.388235,-50.598235
1,2,175.32,152.388235,22.931765
2,3,147.99,152.388235,-4.398235
3,4,152.21,152.388235,-0.178235
4,5,169.36,152.388235,16.971765
5,6,157.78,152.388235,5.391765
6,7,133.87,152.388235,-18.518235
7,8,189.12,152.388235,36.731765
8,9,172.17,152.388235,19.781765
9,10,159.50,152.388235,7.111765


In [141]:
table_names('sql_invoicing')

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments


In [142]:
pd.read_sql("""
SELECT *
FROM clients

""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [150]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12


In [165]:
pd.read_sql("""
SELECT 
    client_id,
    name,
    (SELECT SUM(invoice_total)
    FROM invoices
    WHERE client_id = c.client_id) AS total_sales,
    (SELECT AVG(invoice_total) FROM invoices) AS average,
    (SELECT total_sales - average) AS difference
FROM clients c

""", con)

,client_id,name,total_sales,average,difference
0,1,Vinte,802.89,152.388235,650.501765
1,2,Myworks,101.79,152.388235,-50.598235
2,3,Yadel,705.90,152.388235,553.511765
3,4,Kwideo,NaN,152.388235,NaN
4,5,Topiclounge,980.02,152.388235,827.631765


# Subqueries in the FROM clause

Whenever we use a subquery in a FROM clause we HAVE to give the subquery an alias whether ot not we use that alias. This is required. 

Writing subqueries in the FROM clause of a SELECT statement may make our main query more complex, there is a better way to solve this problem using views, we can take this subquery and store it in our database as a view then we can call that view, we look at it later.

Takeaway: You can write subquery in the FROM clause of the SELECT statement, but reserve it ONLY for simple queries. 

In [171]:
pd.read_sql("""
SELECT *
FROM(
    SELECT 
        client_id,
        name,
        (SELECT SUM(invoice_total)
        FROM invoices
        WHERE client_id = c.client_id) AS total_sales,
        (SELECT AVG(invoice_total) FROM invoices) AS average,
        (SELECT total_sales - average) AS difference
    FROM clients c
) AS sales_summary
""", con)

,client_id,name,total_sales,average,difference
0,1,Vinte,802.89,152.388235,650.501765
1,2,Myworks,101.79,152.388235,-50.598235
2,3,Yadel,705.90,152.388235,553.511765
3,4,Kwideo,NaN,152.388235,NaN
4,5,Topiclounge,980.02,152.388235,827.631765


In [170]:
pd.read_sql("""
SELECT *
FROM(
    SELECT 
        client_id,
        name,
        (SELECT SUM(invoice_total)
        FROM invoices
        WHERE client_id = c.client_id) AS total_sales,
        (SELECT AVG(invoice_total) FROM invoices) AS average,
        (SELECT total_sales - average) AS difference
    FROM clients c
) AS sales_summary
WHERE total_sales IS NOT NULL
""", con)

,client_id,name,total_sales,average,difference
0,1,Vinte,802.89,152.388235,650.501765
1,2,Myworks,101.79,152.388235,-50.598235
2,3,Yadel,705.90,152.388235,553.511765
3,5,Topiclounge,980.02,152.388235,827.631765


# Essential MySQL Functions

## NUMERIC Function

+ ROUND
+ TRUNCATE
+ CEILING
+ FLOOR
+ ABS
+ RAND

https://dev.mysql.com/doc/refman/8.0/en/numeric-functions.html

In [95]:
pd.read_sql("""
SELECT 
    ROUND(5.23)
""", con)

,ROUND(5.23)
0,5.0


In [97]:
pd.read_sql("""
SELECT 
    ROUND(5.23, 1)
""", con)

,"ROUND(5.23, 1)"
0,5.2


In [100]:
pd.read_sql("""
SELECT 
    ROUND(5.23222, 3)
""", con)

,"ROUND(5.23222, 3)"
0,5.232


In [105]:
pd.read_sql("""
SELECT 
    TRUNCATE(5.23333333, 1)
""", con)

,"TRUNCATE(5.23333333, 1)"
0,5.2


In [109]:
pd.read_sql("""
SELECT 
    CEILING(5.73)
""", con)

,CEILING(5.73)
0,6


In [108]:
pd.read_sql("""
SELECT 
    FLOOR(5.23)
""", con)

,FLOOR(5.23)
0,5


In [110]:
pd.read_sql("""
SELECT 
    ABS(-5.23)
""", con)

,ABS(-5.23)
0,5.23


In [114]:
pd.read_sql("""
SELECT 
    RAND()
""", con)

,RAND()
0,0.790311


## STRING Functions

+ LENGTH
+ LOWER
+ UPPER
+ LTRIM
+ RTRIM
+ TRIM 
+ LEFT
+ RIGHT
+ SUBSTRING
+ LOCATE
+ REPLACE
+ CONCAT

https://dev.mysql.com/doc/refman/8.0/en/string-functions.html

In [117]:
pd.read_sql("""
SELECT 
    LENGTH('SKY') AS Len
""", con)

,Len
0,3


In [125]:
pd.read_sql("""
SELECT 
    LOWER('SKY') 
""", con)

,LOWER('SKY')
0,sky


In [126]:
pd.read_sql("""
SELECT 
    UPPER('sky')
""", con)

,UPPER('sky')
0,SKY


In [128]:
pd.read_sql("""
SELECT 
    LTRIM('   SKY') 
""", con)

,LTRIM(' SKY')
0,SKY


In [129]:
pd.read_sql("""
SELECT 
    RTRIM('SKY   ') 
""", con)

,RTRIM('SKY ')
0,SKY


In [130]:
pd.read_sql("""
SELECT 
    TRIM('   SKY   ')
""", con)

,TRIM(' SKY ')
0,SKY


In [134]:
pd.read_sql("""
SELECT 
    LEFT('Kindergarten', 4) 
""", con)

,"LEFT('Kindergarten', 4)"
0,Kind


In [133]:
pd.read_sql("""
SELECT 
    RIGHT('Kindergarten', 6) 
""", con)

,"RIGHT('Kindergarten', 6)"
0,garten


In [135]:
pd.read_sql("""
SELECT 
    SUBSTRING('Kindergarten', 3, 5) 
""", con)

,"SUBSTRING('Kindergarten', 3, 5)"
0,nderg


In [136]:
pd.read_sql("""
SELECT 
    SUBSTRING('Kindergarten', 3) 
""", con)

,"SUBSTRING('Kindergarten', 3)"
0,ndergarten


In [138]:
pd.read_sql("""
SELECT 
    LOCATE('N', 'Kindergarten') 
""", con)

,"LOCATE('N', 'Kindergarten')"
0,3


In [139]:
pd.read_sql("""
SELECT 
    LOCATE('n', 'Kindergarten') 
""", con)

,"LOCATE('n', 'Kindergarten')"
0,3


In [140]:
# Same result because search is not case sensitive

In [141]:
pd.read_sql("""
SELECT 
    LOCATE('q', 'Kindergarten') 
""", con)

,"LOCATE('q', 'Kindergarten')"
0,0


In [143]:
# In other languages usualy we get back -1 but SQL returns back 0 when we search for a character 
# which does not exist in the string

In [145]:
pd.read_sql("""
SELECT 
    LOCATE('garten', 'Kindergarten') 
""", con)

,"LOCATE('garten', 'Kindergarten')"
0,7


In [146]:
pd.read_sql("""
SELECT 
    REPLACE('Kindergarten', 'garten', 'garden') 
""", con)

,"REPLACE('Kindergarten', 'garten', 'garden')"
0,Kindergarden


In [150]:
pd.read_sql("""
SELECT 
    CONCAT(first_name, ' ', last_name) AS "full name"
FROM customers
""", con)

,full name
0,Babara MacCaffrey
1,Ines Brushfield
2,Freddi Boagey
3,Ambur Roseburgh
4,Clemmie Betchley
5,Elka Twiddell
6,Ilene Dowson
7,Thacher Naseby
8,Romola Rumgay
9,Levy Mynett


## DATE Functions

+ NOW
+ CURDATE
+ CURTIME
+ YEAR
+ MONTH
+ DAY
+ HOUR
+ MINUTE
+ SECOND
+ DAYNAME
+ MONTHNAME
+ EXTRACT 

In [154]:
pd.read_sql("""
SELECT NOW()
""", con)

,NOW()
0,2023-04-18 20:29:45


In [161]:
pd.read_sql("""
SELECT YEAR(NOW()) AS year
""", con)

,year
0,2023


In [156]:
pd.read_sql("""
SELECT MONTH(NOW())
""", con)

,MONTH(NOW())
0,4


In [157]:
pd.read_sql("""
SELECT DAY(NOW())
""", con)

,DAY(NOW())
0,18


In [159]:
pd.read_sql("""
SELECT CURDATE()
""", con) 

,CURDATE()
0,2023-04-18


In [160]:
pd.read_sql("""
SELECT CURTIME()
""", con) 

,CURTIME()
0,0 days 20:31:07


In [165]:
pd.read_sql("""
SELECT MINUTE(NOW())
""", con) 

,MINUTE(NOW())
0,32


In [166]:
pd.read_sql("""
SELECT DAYNAME(NOW())
""", con) 

,DAYNAME(NOW())
0,Tuesday


In [167]:
pd.read_sql("""
SELECT MONTHNAME(NOW())
""", con) 

,MONTHNAME(NOW())
0,April


In [169]:
pd.read_sql("""
SELECT EXTRACT(DAY FROM NOW())
""", con) 

,EXTRACT(DAY FROM NOW())
0,18


In [170]:
pd.read_sql("""
SELECT EXTRACT(MONTH FROM NOW())
""", con) 

,EXTRACT(MONTH FROM NOW())
0,4


In [171]:
pd.read_sql("""
SELECT EXTRACT(YEAR FROM NOW())
""", con) 

,EXTRACT(YEAR FROM NOW())
0,2023


In [14]:
con = connector('sql_store')

In [15]:
pd.read_sql("""
SELECT *
FROM orders
WHERE order_date >= '2019-01-01'
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None


In [27]:
pd.read_sql("""
SELECT *
FROM orders
WHERE YEAR(order_date) + 6 = YEAR(NOW())
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,3,8,2017-12-01,1,None,None,NaN
1,4,2,2017-01-22,1,None,None,NaN
2,5,5,2017-08-25,2,,2017-08-26,3.0
3,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0


In [29]:
pd.read_sql("""
SELECT *
FROM orders
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


## FORMATTING Date and Times

+ DATE_FORMAT

https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html

+ TIME_FORMAT



In [176]:
pd.read_sql("""
SELECT 
    order_date,
    DATE_FORMAT(order_date, '%y')
    
FROM orders
""", con) 

,order_date,"DATE_FORMAT(ORDER_DATE, '%y')"
0,2019-01-30,19
1,2018-08-02,18
2,2017-12-01,17
3,2017-01-22,17
4,2017-08-25,17
5,2018-11-18,18
6,2018-09-22,18
7,2018-06-08,18
8,2017-07-05,17
9,2018-04-22,18


In [177]:
pd.read_sql("""
SELECT 
    order_date,
    DATE_FORMAT(order_date, '%Y')
    
FROM orders
""", con) 

,order_date,"DATE_FORMAT(ORDER_DATE, '%Y')"
0,2019-01-30,2019
1,2018-08-02,2018
2,2017-12-01,2017
3,2017-01-22,2017
4,2017-08-25,2017
5,2018-11-18,2018
6,2018-09-22,2018
7,2018-06-08,2018
8,2017-07-05,2017
9,2018-04-22,2018


In [179]:
pd.read_sql("""
SELECT 
    order_date,
    DATE_FORMAT(order_date, '%m')
    
FROM orders
""", con) 

,order_date,"DATE_FORMAT(ORDER_DATE, '%m')"
0,2019-01-30,01
1,2018-08-02,08
2,2017-12-01,12
3,2017-01-22,01
4,2017-08-25,08
5,2018-11-18,11
6,2018-09-22,09
7,2018-06-08,06
8,2017-07-05,07
9,2018-04-22,04


In [186]:
pd.read_sql("""
SELECT 
    order_date,
    DATE_FORMAT(order_date, '%M')
    
FROM orders
""", con) 

,order_date,"DATE_FORMAT(order_date, '%M')"
0,2019-01-30,January
1,2018-08-02,August
2,2017-12-01,December
3,2017-01-22,January
4,2017-08-25,August
5,2018-11-18,November
6,2018-09-22,September
7,2018-06-08,June
8,2017-07-05,July
9,2018-04-22,April


In [187]:
pd.read_sql("""
SELECT 
    order_date,
    DATE_FORMAT(order_date, '%M %d %Y')
    
FROM orders
""", con) 

,order_date,"DATE_FORMAT(order_date, '%M %d %Y')"
0,2019-01-30,January 30 2019
1,2018-08-02,August 02 2018
2,2017-12-01,December 01 2017
3,2017-01-22,January 22 2017
4,2017-08-25,August 25 2017
5,2018-11-18,November 18 2018
6,2018-09-22,September 22 2018
7,2018-06-08,June 08 2018
8,2017-07-05,July 05 2017
9,2018-04-22,April 22 2018


In [204]:
pd.read_sql("""
SELECT 
    order_date,
    TIME_FORMAT(order_date, '%H')
    
FROM orders
""", con) 

,order_date,"TIME_FORMAT(order_date, '%H')"
0,2019-01-30,00
1,2018-08-02,00
2,2017-12-01,00
3,2017-01-22,00
4,2017-08-25,00
5,2018-11-18,00
6,2018-09-22,00
7,2018-06-08,00
8,2017-07-05,00
9,2018-04-22,00


In [209]:
pd.read_sql("""
SELECT TIME_FORMAT (NOW(), '%H:%i %p')
""", con) 

,"TIME_FORMAT (NOW(), '%H:%i %p')"
0,21:01 PM


## CALCULATING Dates and TImes

+ DATE_ADD

+ DATE_SUB

+ DATEDIFF

+ TIME_TO_SEC



In [211]:
pd.read_sql("""
SELECT DATE_ADD(NOW(), INTERVAL 1 DAY)
""", con) 

,"DATE_ADD(NOW(), INTERVAL 1 DAY)"
0,2023-04-19 21:05:43


In [212]:
pd.read_sql("""
SELECT DATE_ADD(NOW(), INTERVAL 1 YEAR)
""", con) 

,"DATE_ADD(NOW(), INTERVAL 1 YEAR)"
0,2024-04-18 21:06:04


In [213]:
pd.read_sql("""
SELECT DATE_ADD(NOW(), INTERVAL -1 DAY)
""", con) 

,"DATE_ADD(NOW(), INTERVAL -1 DAY)"
0,2023-04-17 21:06:17


In [216]:
pd.read_sql("""
SELECT DATE_SUB(NOW(), INTERVAL -1 DAY)
""", con) 

,"DATE_SUB(NOW(), INTERVAL -1 DAY)"
0,2023-04-19 21:06:36


In [217]:
pd.read_sql("""
SELECT DATE_SUB(NOW(), INTERVAL 1 DAY)
""", con) 

,"DATE_SUB(NOW(), INTERVAL 1 DAY)"
0,2023-04-17 21:06:49


In [218]:
pd.read_sql("""
SELECT DATEDIFF('2019-01-05', '2019-01-01')
""", con) 

,"DATEDIFF('2019-01-05', '2019-01-01')"
0,4


In [220]:
pd.read_sql("""
SELECT DATEDIFF('2019-01-01', '2019-01-05')
""", con) 

,"DATEDIFF('2019-01-01', '2019-01-05')"
0,-4


In [225]:
pd.read_sql("""
SELECT DATEDIFF('2019-01-05', '2019-01-01')
""", con) 

,"DATEDIFF('2019-01-05', '2019-01-01')"
0,4


In [226]:
pd.read_sql("""
SELECT TIME_TO_SEC ('09:00')
""", con) 

,TIME_TO_SEC ('09:00')
0,32400


In [227]:
pd.read_sql("""
SELECT TIME_TO_SEC ('09:00') - TIME_TO_SEC('9:02')
""", con)

,TIME_TO_SEC ('09:00') - TIME_TO_SEC('9:02')
0,-120


## IFNULL and COALESCE Functions

In [35]:
pd.read_sql("""
SELECT *
FROM orders""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [32]:
pd.read_sql("""
SELECT 
    order_id,
    shipper_id
FROM orders""", con)

,order_id,shipper_id
0,1,NaN
1,3,NaN
2,4,NaN
3,6,NaN
4,8,NaN
5,9,1.0
6,10,2.0
7,5,3.0
8,2,4.0
9,7,4.0


In [229]:
pd.read_sql("""
SELECT 
    order_id,
    IFNULL(shipper_id, 'Not assigned') AS shipper
FROM orders""", con)

,order_id,shipper
0,1,Not assigned
1,3,Not assigned
2,4,Not assigned
3,6,Not assigned
4,8,Not assigned
5,9,1
6,10,2
7,5,3
8,2,4
9,7,4


In [231]:
pd.read_sql("""
SELECT 
    order_id,
    shipper_id,
    comments
FROM orders""", con)

,order_id,shipper_id,comments
0,1,NaN,None
1,2,4.0,None
2,3,NaN,None
3,4,NaN,None
4,5,3.0,
5,6,NaN,Aliquam erat volutpat. In congue.
6,7,4.0,None
7,8,NaN,"Mauris enim leo, rhoncus sed, vestibulum sit a..."
8,9,1.0,Nulla mollis molestie lorem. Quisque ut erat.
9,10,2.0,None


In [36]:
pd.read_sql("""
SELECT 
    order_id,
    COALESCE(shipper_id, comments, 'Not assigned') AS shipper
FROM orders""", con)

,order_id,shipper
0,1,Not assigned
1,2,4
2,3,Not assigned
3,4,Not assigned
4,5,3
5,6,Aliquam erat volutpat. In congue.
6,7,4
7,8,"Mauris enim leo, rhoncus sed, vestibulum sit a..."
8,9,1
9,10,2


In [38]:
table_names('sql_store')

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [46]:
pd.read_sql("""
SELECT 
    CONCAT(first_name, ' ', last_name) AS customer,
    IFNULL(phone, 'Unknown') AS phone
FROM customers
""", con)

,customer,phone
0,Babara MacCaffrey,781-932-9754
1,Ines Brushfield,804-427-9456
2,Freddi Boagey,719-724-7869
3,Ambur Roseburgh,407-231-8017
4,Clemmie Betchley,Unknown
5,Elka Twiddell,312-480-8498
6,Ilene Dowson,615-641-4759
7,Thacher Naseby,941-527-3977
8,Romola Rumgay,559-181-3744
9,Levy Mynett,404-246-3370


## IF Function 

In [55]:
pd.read_sql("""
SELECT *
FROM orders
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [232]:
pd.read_sql("""
SELECT 
    order_id,
    customer_id,
    'Active' AS status
FROM orders
WHERE order_date >= '2019-01-01'

UNION
SELECT 
     order_id,
    customer_id,
    'Archived' AS status
FROM orders
WHERE order_date < '2019-01-01'

""", con)

,order_id,customer_id,status
0,1,6,Active
1,2,7,Archived
2,3,8,Archived
3,4,2,Archived
4,5,5,Archived
5,6,10,Archived
6,7,2,Archived
7,8,5,Archived
8,9,10,Archived
9,10,6,Archived


In [50]:
# Using IF funstion I achieved the same results as above but with much shorted query: 

In [233]:
pd.read_sql("""
SELECT 
    order_id,
    customer_id, 
    IF (order_date >= '2019-01-01', 'Active', 'Archived') AS status
FROM orders
""", con)

,order_id,customer_id,status
0,1,6,Active
1,2,7,Archived
2,3,8,Archived
3,4,2,Archived
4,5,5,Archived
5,6,10,Archived
6,7,2,Archived
7,8,5,Archived
8,9,10,Archived
9,10,6,Archived


In [58]:
table_names('sql_store')

         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [59]:
pd.read_sql("""
SELECT *
FROM products
""", con)

,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [61]:
pd.read_sql("""
SELECT *
FROM order_items
""", con)

,order_id,product_id,quantity,unit_price
0,1,4,4,3.74
1,2,1,2,9.10
2,2,4,4,1.66
3,2,6,2,2.94
4,3,3,10,9.12
5,4,3,7,6.99
6,4,10,7,6.40
7,5,2,3,9.89
8,6,1,4,8.65
9,6,2,4,3.28


In [62]:
pd.read_sql("""
SELECT *
FROM orders
""", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [82]:
pd.read_sql("""
SELECT 
    product_id,
    name,
    COUNT(*) AS orders,
    IF (COUNT(*) > 1, 'Many times', 'Once') AS frequency
FROM products 
JOIN order_items oi
    USING (product_id)
GROUP BY name, product_id
""", con)

,product_id,name,orders,frequency
0,1,Foam Dinner Plate,3,Many times
1,2,"Pork - Bacon,back Peameal",2,Many times
2,3,"Lettuce - Romaine, Heart",4,Many times
3,4,"Brocolinni - Gaylan, Chinese",2,Many times
4,5,Sauce - Ranch Dressing,2,Many times
5,6,Petit Baguette,2,Many times
6,8,Island Oasis - Raspberry,1,Once
7,9,Longan,1,Once
8,10,Broom - Push,1,Once


## CASE Operator

In [244]:
pd.read_sql("""
SELECT 
    order_id,
    CASE 
        WHEN YEAR(order_date) = '2019' THEN 'Active'
        WHEN YEAR(order_date) = '2018' THEN 'Last Year'
        WHEN YEAR(order_date) < '2018' THEN 'Archived'
        ELSE 'Future'
    END AS category
FROM orders
""", con)

,order_id,category
0,1,Active
1,2,Last Year
2,3,Archived
3,4,Archived
4,5,Archived
5,6,Last Year
6,7,Last Year
7,8,Last Year
8,9,Archived
9,10,Last Year


In [93]:
pd.read_sql("""
SELECT 
    CONCAT(first_name, ' ', last_name) AS customer,
    points,
    CASE
        WHEN points < 2000 THEN 'Bronze'
        WHEN points BETWEEN 2000 AND 3000 THEN 'Silver'
        WHEN points > 3000 THEN 'Gold'
    END AS category
FROM customers
""", con)

,customer,points,category
0,Babara MacCaffrey,2273,Silver
1,Ines Brushfield,947,Bronze
2,Freddi Boagey,2967,Silver
3,Ambur Roseburgh,457,Bronze
4,Clemmie Betchley,3675,Gold
5,Elka Twiddell,3073,Gold
6,Ilene Dowson,1672,Bronze
7,Thacher Naseby,205,Bronze
8,Romola Rumgay,1486,Bronze
9,Levy Mynett,796,Bronze


In [253]:
# another approach

pd.read_sql("""
SELECT 
    CONCAT(first_name, ' ', last_name) AS customer,
    points,
    CASE
        WHEN points > 3000 THEN 'Gold'
        WHEN points >= 2000 THEN 'Silver'
        ELSE 'Bronze'
    END AS category
FROM customers
""", con)

,customer,points,category
0,Babara MacCaffrey,2273,Silver
1,Ines Brushfield,947,Bronze
2,Freddi Boagey,2967,Silver
3,Ambur Roseburgh,457,Bronze
4,Clemmie Betchley,3675,Gold
5,Elka Twiddell,3073,Gold
6,Ilene Dowson,1672,Bronze
7,Thacher Naseby,205,Bronze
8,Romola Rumgay,1486,Bronze
9,Levy Mynett,796,Bronze


# VIEWS

## Creating VIEWS

In [14]:
con = connector('sql_invoicing')

In [15]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None


In [16]:
pd.read_sql("""
SELECT 
    c.client_id
FROM clients c

""", con)

,client_id
0,1
1,2
2,3
3,4
4,5


In [17]:
pd.read_sql("""

SELECT 
    c.client_id,
    c.name,
    SUM(invoice_total) AS total_sales
FROM clients c
JOIN invoices USING (client_id)
GROUP BY client_id, name
""", con)

,client_id,name,total_sales
0,2,Myworks,101.79
1,5,Topiclounge,980.02
2,3,Yadel,705.90
3,1,Vinte,802.89


In [21]:
cursor = con.cursor()

cursor.execute("""
CREATE VIEW sales_by_client AS
SELECT 
    c.client_id,
    c.name,
    SUM(invoice_total) AS total_sales
FROM clients c
JOIN invoices USING (client_id)
GROUP BY client_id, name

""")

con.commit()
cursor.close()

In [27]:
views_names('sql_invoicing')

        TABLE_NAME
0  sales_by_client


In [28]:
pd.read_sql("""
SELECT *
FROM sales_by_client
""", con)

,client_id,name,total_sales
0,2,Myworks,101.79
1,5,Topiclounge,980.02
2,3,Yadel,705.90
3,1,Vinte,802.89


In [30]:
pd.read_sql("""
SELECT *
FROM sales_by_client
ORDER BY total_sales DESC
""", con)

,client_id,name,total_sales
0,5,Topiclounge,980.02
1,1,Vinte,802.89
2,3,Yadel,705.90
3,2,Myworks,101.79


In [31]:
pd.read_sql("""
SELECT *
FROM sales_by_client
WHERE total_sales > 500
""", con)

,client_id,name,total_sales
0,5,Topiclounge,980.02
1,3,Yadel,705.90
2,1,Vinte,802.89


In [32]:
pd.read_sql("""
SELECT *
FROM sales_by_client
JOIN clients USING (client_id)
""", con)

,client_id,name,total_sales,name,address,city,state,phone
0,1,Vinte,802.89,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,101.79,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,705.90,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,5,Topiclounge,980.02,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [33]:
table_names('sql_invoicing')

        TABLE_NAME
0          clients
1         invoices
2  payment_methods
3         payments
4  sales_by_client


In [34]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [36]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12


In [41]:
pd.read_sql("""
SELECT 
    c.client_id,
    c.name,
    SUM(invoice_total - payment_total) AS balance 
FROM clients c
JOIN invoices i
    USING (client_id)
GROUP BY client_id, name
""", con)

,client_id,name,balance
0,2,Myworks,101.79
1,5,Topiclounge,841.63
2,3,Yadel,557.46
3,1,Vinte,728.34


In [44]:
cursor = con.cursor()

cursor.execute("""

CREATE VIEW clients_balance AS
SELECT 
    c.client_id,
    c.name,
    SUM(invoice_total - payment_total) AS balance 
FROM clients c
JOIN invoices i
    USING (client_id)
GROUP BY client_id, name 
""")

con.commit()
cursor.close()

In [40]:
pd.read_sql("""
SELECT *
FROM balance
""", con)

,client_id,name,balance
0,1,Vinte,83.23
1,1,Vinte,189.12
2,1,Vinte,159.50
3,1,Vinte,162.02
4,1,Vinte,134.47
5,2,Myworks,101.79
6,3,Yadel,152.21
7,3,Yadel,133.87
8,3,Yadel,126.12
9,3,Yadel,86.98


In [45]:
pd.read_sql("""
SELECT *
FROM clients_balance
""", con)

,client_id,name,balance
0,2,Myworks,101.79
1,5,Topiclounge,841.63
2,3,Yadel,557.46
3,1,Vinte,728.34


## Altering or Dropping VIEWS

In [48]:
cursor = con.cursor()

cursor.execute("""

CREATE OR REPLACE VIEW clients_balance AS
SELECT 
    c.client_id,
    c.name,
    SUM(invoice_total - payment_total) AS balance 
FROM clients c
JOIN invoices i
    USING (client_id)
GROUP BY client_id, name 
ORDER BY balance DESC
""")

con.commit()
cursor.close()

In [49]:
pd.read_sql("""
SELECT *
FROM clients_balance
""", con)

,client_id,name,balance
0,5,Topiclounge,841.63
1,1,Vinte,728.34
2,3,Yadel,557.46
3,2,Myworks,101.79


## Updating VIEWS

In [53]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,2019-06-11,2019-07-01,2019-02-12


In [54]:
cursor = con.cursor()

cursor.execute("""
CREATE OR REPLACE VIEW invoices_with_balance AS
SELECT 
    invoice_id,
    number,
    client_id,
    invoice_total,
    payment_total,
    invoice_total - payment_total AS balance, 
    invoice_date,
    due_date,
    payment_date
FROM invoices
WHERE (invoice_total - payment_total) > 0
""")

con.commit()
cursor.close()

In [57]:
pd.read_sql("""
SELECT *
FROM invoices_with_balance
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,1,91-953-3396,2,101.79,0.00,101.79,2019-03-09,2019-03-29,None
1,2,03-898-6735,5,175.32,8.18,167.14,2019-06-11,2019-07-01,2019-02-12
2,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
3,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
4,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
5,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
6,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
7,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
8,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
9,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None


Since in my view above i don't have DISTINCT keyword, or using any aggregate functions or groupby or union so my view is UPDATABLE!

In [59]:
cursor = con.cursor()

cursor.execute("""
DELETE FROM invoices_with_balance
WHERE invoice_id = 1

""")

con.commit()
cursor.close()


In [60]:
pd.read_sql("""
SELECT *
FROM invoices_with_balance
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,8.18,167.14,2019-06-11,2019-07-01,2019-02-12
1,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
2,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
3,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
4,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
5,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
6,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
7,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
8,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None
9,11,20-848-0181,3,126.15,0.03,126.12,2019-01-07,2019-01-27,2019-01-11


In [62]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices_with_balance
SET due_date = DATE_ADD(due_date, INTERVAL 2 DAY)
WHERE invoice_id = 2
""")

con.commit()
cursor.close()


In [63]:
pd.read_sql("""
SELECT *
FROM invoices_with_balance
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,8.18,167.14,2019-06-11,2019-07-03,2019-02-12
1,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
2,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
3,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
4,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
5,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
6,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
7,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
8,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None
9,11,20-848-0181,3,126.15,0.03,126.12,2019-01-07,2019-01-27,2019-01-11


## WITH CHECK OPTION

In [64]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices_with_balance
SET payment_total = invoice_total
WHERE invoice_id = 2

""")

con.commit()
cursor.close()

In [65]:
pd.read_sql("""
SELECT *
FROM invoices_with_balance
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
1,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
2,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
3,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
4,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
5,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
6,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
7,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None
8,11,20-848-0181,3,126.15,0.03,126.12,2019-01-07,2019-01-27,2019-01-11
9,13,41-666-1035,5,135.01,87.44,47.57,2019-06-25,2019-07-15,2019-01-26


In [66]:
cursor = con.cursor()

cursor.execute("""
CREATE OR REPLACE VIEW invoices_with_balance AS
SELECT 
    invoice_id,
    number,
    client_id,
    invoice_total,
    payment_total,
    invoice_total - payment_total AS balance, 
    invoice_date,
    due_date,
    payment_date
FROM invoices
WHERE (invoice_total - payment_total) > 0
WITH CHECK OPTION

""")

con.commit()
cursor.close()

In [68]:
cursor = con.cursor()

cursor.execute("""
UPDATE invoices_with_balance
SET payment_total = invoice_total
WHERE invoice_id = 3

""")

con.commit()
cursor.close()

OperationalError: (1369, "CHECK OPTION failed 'sql_invoicing.invoices_with_balance'")

So now I get (1369, "CHECK OPTION failed 'sql_invoicing.invoices_with_balance'") which prevents excluding any row!

# Stored Procedures and Functions

In [17]:
con = connector('sql_invoicing')

In [16]:
table_names('sql_invoicing')

              TABLE_NAME
0                balance
1                clients
2        clients_balance
3               invoices
4  invoices_with_balance
5        payment_methods
6               payments
7        sales_by_client


In [43]:
cursor = con.cursor()

#cursor.execute("DELIMITER $$") # I DO NOT NEED THIS WHEN USING PYTHON

cursor.execute("""

CREATE PROCEDURE get_clients()

BEGIN 
    SELECT * FROM clients; 
END

""")
# cursor.execute("DELIMITER ;") # I DO NOT NEED THIS WHEN USING PYTHON 

con.commit()
cursor.close()

In [44]:
stored_procedures_names('sql_invoicing')

  SPECIFIC_NAME
0   get_clients


In [45]:
pd.read_sql("""
CALL get_clients ()
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [48]:
views_names('sql_invoicing')

              TABLE_NAME
0                balance
1        clients_balance
2  invoices_with_balance
3        sales_by_client


In [49]:
table_names('sql_invoicing')

              TABLE_NAME
0                balance
1                clients
2        clients_balance
3               invoices
4  invoices_with_balance
5        payment_methods
6               payments
7        sales_by_client


In [51]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,175.32,2019-06-11,2019-07-03,2019-02-12
1,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None


In [56]:
cursor = con.cursor()

cursor.execute("""

CREATE PROCEDURE get_invoices_with_balance ()

BEGIN
    SELECT *
    FROM invoices
    WHERE invoice_total - payment_total > 0; 
END 

""")

con.commit()
cursor.close()


In [57]:
pd.read_sql("""
CALL get_invoices_with_balance ()
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
1,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
2,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
3,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
4,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
5,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
6,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
7,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None
8,11,20-848-0181,3,126.15,0.03,2019-01-07,2019-01-27,2019-01-11
9,13,41-666-1035,5,135.01,87.44,2019-06-25,2019-07-15,2019-01-26


In [58]:
# I also can use my view to create the stored procedure, this is better because I also can see the balance column

pd.read_sql("""
SELECT *
FROM invoices_with_balance
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
1,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
2,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
3,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
4,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
5,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
6,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
7,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None
8,11,20-848-0181,3,126.15,0.03,126.12,2019-01-07,2019-01-27,2019-01-11
9,13,41-666-1035,5,135.01,87.44,47.57,2019-06-25,2019-07-15,2019-01-26


In [59]:
cursor = con.cursor()

cursor.execute("""

CREATE PROCEDURE get_invoices_with_balance_from_view ()

BEGIN
    SELECT *
    FROM invoices_with_balance 
    WHERE balance > 0; 
END
""")

con.commit()
cursor.close()


In [60]:
pd.read_sql("""
CALL get_invoices_with_balance_from_view ()
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,balance,invoice_date,due_date,payment_date
0,3,20-228-0335,5,147.99,0.00,147.99,2019-07-31,2019-08-20,None
1,4,56-934-0748,3,152.21,0.00,152.21,2019-03-08,2019-03-28,None
2,5,87-052-3121,5,169.36,0.00,169.36,2019-07-18,2019-08-07,None
3,6,75-587-6626,1,157.78,74.55,83.23,2019-01-29,2019-02-18,2019-01-03
4,7,68-093-9863,3,133.87,0.00,133.87,2019-09-04,2019-09-24,None
5,8,78-145-1093,1,189.12,0.00,189.12,2019-05-20,2019-06-09,None
6,9,77-593-0081,5,172.17,0.00,172.17,2019-07-09,2019-07-29,None
7,10,48-266-1517,1,159.50,0.00,159.50,2019-06-30,2019-07-20,None
8,11,20-848-0181,3,126.15,0.03,126.12,2019-01-07,2019-01-27,2019-01-11
9,13,41-666-1035,5,135.01,87.44,47.57,2019-06-25,2019-07-15,2019-01-26


## Parameters

In [62]:
pd.read_sql("""
SELECT *
FROM clients
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [71]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_clients_by_state")

cursor.execute("""

CREATE PROCEDURE get_clients_by_state 
(
state CHAR(2)
)
BEGIN
    SELECT *
    FROM clients c
    WHERE c.state = state; 
END
""")

con.commit()
cursor.close()


In [73]:
pd.read_sql("""
CALL get_clients_by_state('CA')
""", con)

,client_id,name,address,city,state,phone
0,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037


In [77]:
pd.read_sql("""
SELECT *
FROM invoices
LIMIT 2
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,175.32,2019-06-11,2019-07-03,2019-02-12
1,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None


In [78]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_invoices_by_client")

cursor.execute("""

CREATE PROCEDURE get_invoices_by_client
(
client_id INT
)
BEGIN
    SELECT *
    FROM invoices i
    WHERE i.client_id = client_id; 
END 
""")

con.commit()
cursor.close()


In [81]:
pd.read_sql("""
CALL get_invoices_by_client (1)
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
1,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
2,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None
3,16,10-451-8824,1,162.02,0.00,2019-03-30,2019-04-19,None
4,19,83-559-4105,1,134.47,0.00,2019-11-23,2019-12-13,None


## Parameters with default values

In [82]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_clients_by_state")

cursor.execute("""
CREATE PROCEDURE get_clients_by_state
(
state CHAR(2)
)
BEGIN
    IF state IS NULL THEN
        SET state = 'CA';
    END IF; 
    
    SELECT *
    FROM clients c
    WHERE c.state = state; 
END
""")

con.commit()
cursor.close()



In [83]:
pd.read_sql("""
CALL get_clients_by_state ('CA')
""", con)

,client_id,name,address,city,state,phone
0,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037


In [85]:
pd.read_sql("""
CALL get_clients_by_state (NULL)
""", con)

,client_id,name,address,city,state,phone
0,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037


In [89]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_clients_by_state")

cursor.execute("""

CREATE PROCEDURE get_clients_by_state
(
state CHAR(2)
)
BEGIN
    IF state IS NULL THEN
        SELECT * FROM clients;
    ELSE 
        SELECT * FROM clients c 
        WHERE c.state = state; 
    END IF; 
END

""")

con.commit()
cursor.close()



In [93]:
pd.read_sql("""
CALL get_clients_by_state(NULL)
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [94]:
pd.read_sql("""
CALL get_clients_by_state('CA')
""", con)

,client_id,name,address,city,state,phone
0,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037


In [95]:
# BUT the above approach is a bit verbose and amateurous the better way is as follows: 

In [96]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_clients_by_state")

cursor.execute("""

CREATE PROCEDURE get_clients_by_state
(
state CHAR(2)
)
BEGIN
    SELECT * FROM clients c 
    WHERE c.state = IFNULL(state, c.state); 
END

""")

con.commit()
cursor.close()



In [99]:
pd.read_sql("""
CALL get_clients_by_state(NULL)
""", con)

,client_id,name,address,city,state,phone
0,1,Vinte,3 Nevada Parkway,Syracuse,NY,315-252-7305
1,2,Myworks,34267 Glendale Parkway,Huntington,WV,304-659-1170
2,3,Yadel,096 Pawling Parkway,San Francisco,CA,415-144-6037
3,4,Kwideo,81674 Westerfield Circle,Waco,TX,254-750-0784
4,5,Topiclounge,0863 Farmco Road,Portland,OR,971-888-9129


In [103]:
table_names('sql_invoicing')

              TABLE_NAME
0                balance
1                clients
2        clients_balance
3               invoices
4  invoices_with_balance
5        payment_methods
6               payments
7        sales_by_client


In [128]:
pd.read_sql("""
SELECT *
FROM payments
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [140]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS get_payments")
cursor.execute("""

CREATE PROCEDURE get_payments
(
    client_id INT, 
    payment_method_id TINYINT
)
BEGIN 
    SELECT *
    FROM payments p
    WHERE p.client_id = IFNULL (client_id, p.client_id) AND
          p.payment_method = IFNULL (payment_method_id, p.payment_method); 
END
""")

con.commit()
cursor.close()

In [141]:
pd.read_sql("""
CALL get_payments (NULL, NULL)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,2,1,6,2019-01-03,74.55,1
2,3,3,11,2019-01-11,0.03,1
3,4,5,13,2019-01-26,87.44,1
4,5,3,15,2019-01-15,80.31,1
5,6,3,17,2019-01-15,68.10,1
6,7,5,18,2019-01-08,32.77,1
7,8,5,18,2019-01-08,10.00,2


In [135]:
pd.read_sql("""
CALL get_payments (1, NULL)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,2,1,6,2019-01-03,74.55,1


In [136]:
pd.read_sql("""
CALL get_payments (5, NULL)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,1,5,2,2019-02-12,8.18,1
1,4,5,13,2019-01-26,87.44,1
2,7,5,18,2019-01-08,32.77,1
3,8,5,18,2019-01-08,10.00,2


In [137]:
pd.read_sql("""
CALL get_payments (5, 2)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,8,5,18,2019-01-08,10.0,2


In [138]:
pd.read_sql("""
CALL get_payments (5, 3)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method


In [139]:
pd.read_sql("""
CALL get_payments (NULL, 2)
""", con)

,payment_id,client_id,invoice_id,date,amount,payment_method
0,8,5,18,2019-01-08,10.0,2


## Parameter Validation

In [13]:
con =connector('sql_invoicing')

In [ ]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS make_payment")
cursor.execute("""

CREATE PROCEDURE make_payment
(
    invoice_id INT,
    payment_amount DECIMAL(9, 2),
    payment_date DATE
    )


BEGIN
    UPDATE invoices i
    SET 
        i.payment_total = payment_amount, 
        i.payment_date = payment_date 
    WHERE i.invoice_id = invoice_id;     

END
""")

con.commit()
cursor.close()

In [26]:
cursor = con.cursor()

cursor.execute("""
CALL make_payment (2, 100, '2019-01-01')
""")

con.commit()
cursor.close()

In [27]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,100.00,2019-06-11,2019-07-03,2019-01-01
1,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
2,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
3,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
4,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
5,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
6,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
7,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
8,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None
9,11,20-848-0181,3,126.15,0.03,2019-01-07,2019-01-27,2019-01-11


In [28]:
cursor = con.cursor()

cursor.execute("""
CALL make_payment (2, -100, '2019-01-01')
""")

con.commit()
cursor.close()

In [29]:
pd.read_sql("""
SELECT *
FROM invoices
""", con)

,invoice_id,number,client_id,invoice_total,payment_total,invoice_date,due_date,payment_date
0,2,03-898-6735,5,175.32,-100.00,2019-06-11,2019-07-03,2019-01-01
1,3,20-228-0335,5,147.99,0.00,2019-07-31,2019-08-20,None
2,4,56-934-0748,3,152.21,0.00,2019-03-08,2019-03-28,None
3,5,87-052-3121,5,169.36,0.00,2019-07-18,2019-08-07,None
4,6,75-587-6626,1,157.78,74.55,2019-01-29,2019-02-18,2019-01-03
5,7,68-093-9863,3,133.87,0.00,2019-09-04,2019-09-24,None
6,8,78-145-1093,1,189.12,0.00,2019-05-20,2019-06-09,None
7,9,77-593-0081,5,172.17,0.00,2019-07-09,2019-07-29,None
8,10,48-266-1517,1,159.50,0.00,2019-06-30,2019-07-20,None
9,11,20-848-0181,3,126.15,0.03,2019-01-07,2019-01-27,2019-01-11


In [32]:
cursor = con.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS make_payment")

cursor.execute("""
CREATE PROCEDURE make_payment
(
    invoice_id INT,
    payment_amount DECIMAL(9, 2),
    payment_date DATE
)
BEGIN
    
    IF payment_amount <= 0 THEN 
        SIGNAL SQLSTATE '22003' 
            SET MESSAGE_TEXT = 'Invalid payment amount'; 
    END IF; 
    
    UPDATE invoices i
        SET i.payment_total = payment_amount,
            i.payment_date = payment_date
    WHERE i.invoice_id = invoice_id; 
END
""")

con.commit()
cursor.close()


In [33]:
cursor = con.cursor()

cursor.execute("""
CALL make_payment (2, -100, '2019-01-01')
""")

con.commit()
cursor.close()

OperationalError: (1644, 'Invalid payment amount')